In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time 
import os, sys
import copy

In [ ]:
# If you have clone the repository and open this notebook,
# this notebook should be in likelihood-implementation/notebooks
# Let's set the working directory to be likelihood-implementation

likelihood_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
sys.path.insert(0, likelihood_path)
print('Setting as working directory: ', likelihood_path)

### Check if running on correct environment

In [ ]:
print (os.environ['CONDA_DEFAULT_ENV'])

In [ ]:
# Matplotlib params set-up

%matplotlib inline
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('font',size=25)
plt.rc('axes', titlesize=26)
plt.rc('axes', labelsize=25)
plt.rc('lines', linewidth=2)
plt.rc('lines', markersize=6)
plt.rc('legend', fontsize=20)
plt.rc('mathtext', fontset='stix')
plt.rc('font', family='STIXGeneral')

In [ ]:
# Import external loglike from the Likelihood Package within cobaya_interface.py

from cloe.cobaya_interface import EuclidLikelihood

In [ ]:
# We are running the Euclid-Likelihood as an external likelihood class for Cobaya
# Cobaya needs a dictionary or yaml file as input to start running
# This dictionary below ('info') can be modified up to some point by the user to
# adapt it to the user's needs.
# The options that can be modified by the user are pointed with the acronym (UC).

info = {
    #'params': Cobaya's protected key of the input dictionary. 
    # Includes the parameters that the user would like to sample over:
'params': {
        # (UC): each parameter below (which is a 'key' of another sub-dictionary) can contain a dictionary
        # with the key 'prior', 'latex'...
        # If the prior dictionary is not passed to a parameter, this parameter is fixed.
        # In this example, we are sampling the parameter ns
        # For more information see: https://cobaya.readthedocs.io/en/latest/example.html
        'ombh2': 0.022445, #Omega density of baryons times the reduced Hubble parameter squared
        'omch2': 0.1205579307, #Omega density of cold dark matter times the reduced Hubble parameter squared
        'H0': 67, #Hubble parameter evaluated today (z=0) in km/s/Mpc
        'tau': 0.0925, #optical depth
        'mnu': 0.06, #  sum of the mass of neutrinos in eV
        'nnu': 3.046, #N_eff of relativistic species 
        'As': 2.12605e-9, #Amplitude of the primordial scalar power spectrum
        'ns': 0.96, # primordial power spectrum tilt
        'w': -1, #Dark energy fluid model
        'wa': 0, #Dark energy fluid model
        'omk': 0.0, #curvature density
        #'omegam': None, #DERIVED parameter: Omega matter density
        #'omegab': None, #DERIVED parameter: Omega baryon density
        #'omeganu': None, #DERIVED parameter: Omega neutrino density
        #'omnuh2': None, #DERIVED parameter: Omega neutrino density times de reduced Hubble parameter squared
        #'omegac': None, #DERIVED parameter: Omega cold dark matter density
        #'N_eff': None,
        # (UC): baryonic feedback for built-in models (Mead2016/Mead2020_feedback; see next cell)
        # These parameters are considered only when the corresponding nonlinear modelling option is selected:
        # HMCode_A_baryon: 3.13 # default value for Mead2016
        # HMCode_eta_baryon: 0.603 # default value for Mead2016
        # HMCode_logT_AGN: 7.8 # default value for Mead2020_feedback
        # (UC): galaxy bias parameters:
        # The bias parameters below are currently fixed to the
        # values used by the Inter Science Taskforce: Forecast (IST:F)
        # and presented in the corresponding IST:F paper (arXiv: 1910.09273).
        # However, they can be changed by the user and even sample over them by putting a prior
        # Photometric bias parameters (defaults for 10 bins)
        'b1_photo': 1.0997727037892875,
        'b2_photo': 1.220245876862528,
        'b3_photo': 1.2723993083933989,
        'b4_photo': 1.316624471897739,
        'b5_photo': 1.35812370570578,
        'b6_photo': 1.3998214171814918,
        'b7_photo': 1.4446452851824907,
        'b8_photo': 1.4964959071110084,
        'b9_photo': 1.5652475842498528,
        'b10_photo': 1.7429859437184225,
        # Adding these here just for generation of params.yaml
        'b11_photo': 1.7429859437184225,
        'b12_photo': 1.7429859437184225,
        'b13_photo': 1.7429859437184225,
        # These values are the defaults for 13 bins 
        #'b1_photo': 1.03047,
        #'b2_photo': 1.06699,
        #'b3_photo': 1.17363,
        #'b4_photo': 1.23340,
        #'b5_photo': 1.27510,
        #'b6_photo': 1.28574,
        #'b7_photo': 1.39434,
        #'b8_photo': 1.49170,
        #'b9_photo': 1.52334,
        #'b10_photo': 1.54639,
        #'b11_photo': 1.68682,
        #'b12_photo': 2.03066,
        #'b13_photo': 2.57812,
        # Bias parameters for polynomial model
        'b0_poly_photo': 0.830703,
        'b1_poly_photo': 1.190547,
        'b2_poly_photo': -0.928357,
        'b3_poly_photo': 0.423292,
        # Magnification bias parameters
        'magnification_bias_1': -0.8499,
        'magnification_bias_2': -1.023,
        'magnification_bias_3': -0.9474,
        'magnification_bias_4': -0.9837,
        'magnification_bias_5': -0.7699,
        'magnification_bias_6': -0.7137,
        'magnification_bias_7': -0.52345,
        'magnification_bias_8': -0.40845,
        'magnification_bias_9': -0.41045,
        'magnification_bias_10': -0.0824,
        'magnification_bias_11': 0.4917,
        'magnification_bias_12': 0.98555,
        'magnification_bias_13': 1.6056,
        # Shear calibration multiplicative bias parameters                                                                                                                                                                                                                                                                                                            
        'multiplicative_bias_1': 0.0,
        'multiplicative_bias_2': 0.0,
        'multiplicative_bias_3': 0.0,
        'multiplicative_bias_4': 0.0,
        'multiplicative_bias_5': 0.0,
        'multiplicative_bias_6': 0.0,
        'multiplicative_bias_7': 0.0,
        'multiplicative_bias_8': 0.0,
        'multiplicative_bias_9': 0.0,
        'multiplicative_bias_10': 0.0,
        'multiplicative_bias_11': 0.0,
        'multiplicative_bias_12': 0.0,
        'multiplicative_bias_13': 0.0,
        ############################
        # Spectroscopic parameters #
        ############################
        # Linear bias #
        'b1_spectro_bin1': 1.46, 'b1_spectro_bin2': 1.61, 'b1_spectro_bin3': 1.75, 'b1_spectro_bin4': 1.90,
        # Local quadratic bias #
        'b2_spectro_bin1': 0.0, 'b2_spectro_bin2': 0.0, 'b2_spectro_bin3': 0.0, 'b2_spectro_bin4': 0.0,
        # FOG counterterms #
        'c0_spectro_bin1': 0.0, 'c0_spectro_bin2': 0.0, 'c0_spectro_bin3': 0.0, 'c0_spectro_bin4': 0.0,
        'c2_spectro_bin1': 0.0, 'c2_spectro_bin2': 0.0, 'c2_spectro_bin3': 0.0, 'c2_spectro_bin4': 0.0,
        'c4_spectro_bin1': 0.0, 'c4_spectro_bin2': 0.0, 'c4_spectro_bin3': 0.0, 'c4_spectro_bin4': 0.0,
        # Deviations from Poisson shot-noise #
        'aP_spectro_bin1': 0.0, 'aP_spectro_bin2': 0.0, 'aP_spectro_bin3': 0.0, 'aP_spectro_bin4': 0.0,
        # Inverse number density #
        'Psn_spectro_bin1': 0.0, 'Psn_spectro_bin2': 0.0, 'Psn_spectro_bin3': 0.0, 'Psn_spectro_bin4': 0.0,
        # Intrinsic alignment parameters
        'a1_ia': 1.72,
        'a2_ia': 2,
        'b1_ia': 1,
        'eta1_ia': -0.41,
        'eta2_ia': 1,
        'beta1_ia': 0.0,
        'pivot_redshift': 0.0,
        # Redshift distributions nuisance parameters: shifts
        'dz_1_GCphot': 0., 'dz_1_WL': 0.,
        'dz_2_GCphot': 0., 'dz_2_WL': 0.,
        'dz_3_GCphot': 0., 'dz_3_WL': 0.,
        'dz_4_GCphot': 0., 'dz_4_WL': 0.,
        'dz_5_GCphot': 0., 'dz_5_WL': 0.,
        'dz_6_GCphot': 0., 'dz_6_WL': 0.,
        'dz_7_GCphot': 0., 'dz_7_WL': 0.,
        'dz_8_GCphot': 0., 'dz_8_WL': 0.,
        'dz_9_GCphot': 0., 'dz_9_WL': 0.,
        'dz_10_GCphot': 0., 'dz_10_WL': 0.,
        'dz_11_GCphot': 0., 'dz_11_WL': 0.,
        'dz_12_GCphot': 0., 'dz_12_WL': 0.,
        'dz_13_GCphot': 0., 'dz_13_WL': 0.,
        ###############################################################################
        # Baryonic parameters
        # Two emulators have different parameters and different ways of setting them:
        #  - Parameters can be set for each redshift bin as par_i, with i= 1, ..., 13  (currently max 13 bins)
        #  - Or as a power law in redshift par(z) = par_0 * (1+z)^(-nu_par)
        ###############################################################################
        # Example BCemu baryon params (from fit to BAHAMAS for 10 bins, values for bins 11 - 13 are the same as bin 10)
        'log10Mc_bcemu_bin1': 13.402342835406102, 'log10Mc_bcemu_bin2': 13.255603572838046, 'log10Mc_bcemu_bin3': 13.39534479993441,
        'log10Mc_bcemu_bin4': 13.491189798802624, 'log10Mc_bcemu_bin5': 13.517099731729203, 'log10Mc_bcemu_bin6': 13.431948448938623,
        'log10Mc_bcemu_bin7': 13.159142065726643, 'log10Mc_bcemu_bin8': 12.630530198678242, 'log10Mc_bcemu_bin9': 11.768928839012311,
        'log10Mc_bcemu_bin10': 11.123969034839764,'log10Mc_bcemu_bin11': 11.123969034839764,'log10Mc_bcemu_bin12': 11.123969034839764,
        'log10Mc_bcemu_bin13': 11.123969034839764,
        'mu_bcemu_bin1': 1.081169575215787, 'mu_bcemu_bin2': 1.0748796706532235, 'mu_bcemu_bin3': 1.257505674386812, 
        'mu_bcemu_bin4': 1.3953104854720346, 'mu_bcemu_bin5': 1.4707848100185459, 'mu_bcemu_bin6': 1.451326233210004, 
        'mu_bcemu_bin7': 1.2738670043792995, 'mu_bcemu_bin8': 0.8851923901330802, 'mu_bcemu_bin9': 0.23572246186875243, 
        'mu_bcemu_bin10': -0.006953332782079408, 'mu_bcemu_bin11': -0.006953332782079408, 'mu_bcemu_bin12': -0.006953332782079408,
        'mu_bcemu_bin13': -0.006953332782079408, 
        'thej_bcemu_bin1': 3.6711008267835776, 'thej_bcemu_bin2': 4.0853297524846175, 'thej_bcemu_bin3': 4.1001870114090835,
        'thej_bcemu_bin4': 4.085897535238355, 'thej_bcemu_bin5': 4.062128106370927, 'thej_bcemu_bin6': 4.0416849190138695, 
        'thej_bcemu_bin7': 4.0405617133891685, 'thej_bcemu_bin8': 4.073797196972992, 'thej_bcemu_bin9': 4.168188042786498, 
        'thej_bcemu_bin10': 4.567495847826163, 'thej_bcemu_bin11': 4.567495847826163, 'thej_bcemu_bin12': 4.567495847826163, 
        'thej_bcemu_bin13': 4.567495847826163,
        'gamma_bcemu_bin1': 2.732738146480909, 'gamma_bcemu_bin2': 2.794658925091356, 'gamma_bcemu_bin3': 3.008141783989797,
        'gamma_bcemu_bin4': 3.2186226058208787, 'gamma_bcemu_bin5': 3.425034377127418, 'gamma_bcemu_bin6': 3.6237277892257937, 
        'gamma_bcemu_bin7': 3.805233587893605, 'gamma_bcemu_bin8': 3.952453083293017, 'gamma_bcemu_bin9': 4.012632091622541,
        'gamma_bcemu_bin10': 3.1454520255604006, 'gamma_bcemu_bin11': 3.1454520255604006, 'gamma_bcemu_bin12': 3.1454520255604006,
        'gamma_bcemu_bin13': 3.1454520255604006,
        'delta_bcemu_bin1': 6.407442578598766, 'delta_bcemu_bin2': 7.0846518716943025, 'delta_bcemu_bin3': 7.226700228338714,
        'delta_bcemu_bin4': 7.343194626402527, 'delta_bcemu_bin5': 7.470370166898234, 'delta_bcemu_bin6': 7.638743359373677,
        'delta_bcemu_bin7': 7.891612514615537, 'delta_bcemu_bin8': 8.252389998618296, 'delta_bcemu_bin9': 8.702184867164187,
        'delta_bcemu_bin10': 7.994110032082859, 'delta_bcemu_bin11': 7.994110032082859, 'delta_bcemu_bin12': 7.994110032082859,
        'delta_bcemu_bin13': 7.994110032082859,
        'eta_bcemu_bin1': 0.20680675958346154, 'eta_bcemu_bin2': 0.22813237994091348, 'eta_bcemu_bin3': 0.21989900653826788,
        'eta_bcemu_bin4': 0.2090499814223423, 'eta_bcemu_bin5': 0.19645554090162448, 'eta_bcemu_bin6': 0.18238365514915927, 
        'eta_bcemu_bin7': 0.16686344766458094, 'eta_bcemu_bin8': 0.14883503959461816, 'eta_bcemu_bin9': 0.1240089652119986,
        'eta_bcemu_bin10': 0.044931761090432315, 'eta_bcemu_bin11': 0.044931761090432315, 'eta_bcemu_bin12': 0.044931761090432315,
        'eta_bcemu_bin13': 0.044931761090432315,
        'deta_bcemu_bin1': 0.08189803795251066, 'deta_bcemu_bin2': 0.05028436917683151, 'deta_bcemu_bin3': 0.04842000359519622,
        'deta_bcemu_bin4': 0.04869423933122641, 'deta_bcemu_bin5': 0.04959595282843388, 'deta_bcemu_bin6': 0.05011878848954406,
        'deta_bcemu_bin7': 0.04907477885822071, 'deta_bcemu_bin8': 0.04681178740273659, 'deta_bcemu_bin9': 0.048062862071344824,
        'deta_bcemu_bin10': 0.14550946518064195, 'deta_bcemu_bin11': 0.14550946518064195, 'deta_bcemu_bin12': 0.14550946518064195,
        'deta_bcemu_bin13': 0.14550946518064195,
        # BCemu params for the redshift parametrization
        # To mimic the BCemu 3-parameter model, just vary the first 6 params and leave the others fixed to these values.
        # A 1-parameter model would use just log10Mc (and others fixed to current values)
        'log10Mc_bcemu_0': 13.32,
        'nu_log10Mc_bcemu': -0.15,
        'thej_bcemu_0': 3.5,
        'nu_thej_bcemu': 0.0,
        'deta_bcemu_0': 0.2,
        'nu_deta_bcemu': 0.6,
        'mu_bcemu_0': 1.0,
        'nu_mu_bcemu': 0.0,
        'gamma_bcemu_0': 2.5,
        'nu_gamma_bcemu': 0.0,
        'delta_bcemu_0': 7.0,
        'nu_delta_bcemu': 0.0,
        'eta_bcemu_0': 0.2,
        'nu_eta_bcemu': 0.0,
        # Example Bacco baryon parameters (from fit to BAHAMAS for 10 bins, values for bins 11 - 13 are the same as bin 10)
        'M_c_bacco_bin1': 14.158079090897681,'M_c_bacco_bin2': 14.413536974582364,
        'M_c_bacco_bin3': 14.513655381066439,
        'M_c_bacco_bin4': 14.59501140993621,
        'M_c_bacco_bin5': 14.667602932620955,
        'M_c_bacco_bin6': 14.736117785720888,
        'M_c_bacco_bin7': 14.803902874910724,
        'M_c_bacco_bin8': 14.873354165369339,
        'M_c_bacco_bin9': 14.947678648592403,
        'M_c_bacco_bin10': 15.179943239223197,
        'M_c_bacco_bin11': 15.179943239223197,
        'M_c_bacco_bin12': 15.179943239223197,
        'M_c_bacco_bin13': 15.179943239223197,
        'eta_bacco_bin1': -0.34397323435383403, 'eta_bacco_bin2': -0.3823001659654948,
        'eta_bacco_bin3': -0.3666759403458338,
        'eta_bacco_bin4': -0.34766555599217897,
        'eta_bacco_bin5': -0.32723937222645033,
        'eta_bacco_bin6': -0.3066851485202732,
        'eta_bacco_bin7': -0.28793191687619096,
        'eta_bacco_bin8': -0.2761849439738153,
        'eta_bacco_bin9': -0.29044476508897765,
        'eta_bacco_bin10': -0.3012527587604943,
        'eta_bacco_bin11': -0.3012527587604943,
        'eta_bacco_bin12': -0.3012527587604943,
        'eta_bacco_bin13': -0.3012527587604943,
        'beta_bacco_bin1': -0.1401597529600361,'beta_bacco_bin2': -0.14602731261184787,
        'beta_bacco_bin3': -0.15945179021822595,
        'beta_bacco_bin4': -0.1717165402260191,
        'beta_bacco_bin5': -0.18260340867450087,
        'beta_bacco_bin6': -0.1914516575772147,
        'beta_bacco_bin7': -0.19665359534532956,
        'beta_bacco_bin8': -0.1938633002915493,
        'beta_bacco_bin9': -0.16799956045034847,
        'beta_bacco_bin10': -0.1180328507054961,
        'beta_bacco_bin11': -0.1180328507054961,
        'beta_bacco_bin12': -0.1180328507054961,
        'beta_bacco_bin13': -0.1180328507054961,
        'M1_z0_cen_bacco_bin1': 11.340025775544985,'M1_z0_cen_bacco_bin2': 11.30961906881156,
        'M1_z0_cen_bacco_bin3': 11.366002025818862,
        'M1_z0_cen_bacco_bin4': 11.423530231429538,
        'M1_z0_cen_bacco_bin5': 11.479266623369345,
        'M1_z0_cen_bacco_bin6': 11.529947744472405,
        'M1_z0_cen_bacco_bin7': 11.569052829475833,
        'M1_z0_cen_bacco_bin8': 11.578833725581436,
        'M1_z0_cen_bacco_bin9': 11.496367669453079,
        'M1_z0_cen_bacco_bin10': 11.360811343412026,
        'M1_z0_cen_bacco_bin11': 11.360811343412026,
        'M1_z0_cen_bacco_bin12': 11.360811343412026,
        'M1_z0_cen_bacco_bin13': 11.360811343412026,
        'theta_inn_bacco_bin1': -0.9790217925155928, 'theta_inn_bacco_bin2': -1.3854647301685985,
        'theta_inn_bacco_bin3': -1.3604452515853098,
        'theta_inn_bacco_bin4': -1.2949667258085438,
        'theta_inn_bacco_bin5': -1.2056425079533284,
        'theta_inn_bacco_bin6': -1.0991845011332575,
        'theta_inn_bacco_bin7': -0.9805620838281162,
        'theta_inn_bacco_bin8': -0.8624918408856748,
        'theta_inn_bacco_bin9': -0.7968588249755786,
        'theta_inn_bacco_bin10': -0.5312787510657424,
        'theta_inn_bacco_bin11': -0.5312787510657424,
        'theta_inn_bacco_bin12': -0.5312787510657424,
        'theta_inn_bacco_bin13': -0.5312787510657424,
        'M_inn_bacco_bin1': 12.472,'M_inn_bacco_bin2': 11.098133175636903,
        'M_inn_bacco_bin3': 10.685545014634902,
        'M_inn_bacco_bin4': 10.379401082211185,
        'M_inn_bacco_bin5': 10.124549632511826,
        'M_inn_bacco_bin6': 9.89485323082124,
        'M_inn_bacco_bin7': 9.669674533440968,
        'M_inn_bacco_bin8': 9.423522683043274,
        'M_inn_bacco_bin9': 9.086255671995564,
        'M_inn_bacco_bin10': 8.119132619254136,
        'M_inn_bacco_bin11': 8.119132619254136,
        'M_inn_bacco_bin12': 8.119132619254136,
        'M_inn_bacco_bin13': 8.119132619254136,
        'theta_out_bacco_bin1': 0.2672123790126238,'theta_out_bacco_bin2': 0.21845633941784046,
        'theta_out_bacco_bin3': 0.22229391886311567,
        'theta_out_bacco_bin4': 0.23124624733764712,
        'theta_out_bacco_bin5': 0.24338032280596764,
        'theta_out_bacco_bin6': 0.25802109222979985,
        'theta_out_bacco_bin7': 0.27483978056155306,
        'theta_out_bacco_bin8': 0.29287229590732955,
        'theta_out_bacco_bin9': 0.3076030069491944,
        'theta_out_bacco_bin10': 0.3569278707998519,
        'theta_out_bacco_bin11': 0.3569278707998519,
        'theta_out_bacco_bin12': 0.3569278707998519,
        'theta_out_bacco_bin13': 0.3569278707998519,
        # Bacco params for the redshift parametrization
        # Here too, one can use just the first 6 params to get a simpler model, with the others fixed.
        # Note that the redshift dependence has not been tested by the Bacco collaboration.
        'M_c_bacco_0': 13.25,
        'nu_M_c_bacco': -0.15,
        'eta_bacco_0': -0.3,
        'nu_eta_bacco': 0.0,
        'beta_bacco_0': -0.22,
        'nu_beta_bacco': 0.0,
        'M1_z0_cen_bacco_0': 10.5,
        'nu_M1_z0_cen_bacco': 0.0,
        'theta_inn_bacco_0': -0.86,
        'nu_theta_inn_bacco': 0.0,
        'M_inn_bacco_0': 13.4,
        'nu_M_inn_bacco': 0.0,
        'theta_out_bacco_0': 0.25,
        'nu_theta_out_bacco': 0.0,
        # Growth index
        'gamma_MG': 0.55,
        # Purity 
        'f_out':0.0,
        'f_out_1':0.0,
        'f_out_2': 0.0,
        'f_out_3': 0.0,
        'f_out_4': 0.0,
        'sigma_z': 0.002},
    #'theory': Cobaya's protected key of the input dictionary.
    # Cobaya needs to ask some minimum theoretical requirements to a Boltzman Solver
    # (UC): you can choose between CAMB or CLASS
    # In this DEMO, we use CAMB and specify some CAMB arguments
    # such as the number of massive neutrinos
    # and the dark energy model
    #
    # ATTENTION: If you have CAMB/CLASS already installed and 
    # you are not using the likelihood conda environment 
    # or option (2) in cell (3) (Cobaya modules), you can add an extra key called 'path' within the camb dictionary
    # to point to your already installed CAMB code
    # NOTE: for values of the nonlinear flag larger than 0, a new key is added in info['theory']['camb']['extra_args'],
    # i.e. 'halofit_version', which contains the requested version of halofit, as described above
    'theory': {'camb': 
               {'stop_at_error': True, 
                'extra_args':{'num_massive_neutrinos': 1,
                              'dark_energy_model': 'ppf'}}},
    #'sampler': Cobaya's protected key of the input dictionary.
    # (UC): you can choose the sampler you want to use.
    # Check Cobaya's documentation to see the list of available samplers
    # In this DEMO, we use the 'evaluate' sampler to make a single computation of the posterior distributions
    # Note: if you want to run a simple MCMC sampling choose 'mcmc'
    'sampler': {'evaluate': None},  
    # 'packages_path': Cobaya's protected key of the input dictionary.
    # This is the variable you need to update
    # if you are running Cobaya with cobaya_modules (option (2) above).
    # If you are using the conda likelihood environment or option (1),
    # please, keep the line below commented
    #
    #'packages_path': modules_path,
    #
    #'output': Cobaya's protected key of the input dictionary.
    # Where are the results going to be stored, in case that the sampler produce output files? 
    # For example: chains...
    # (UC): modify the path below within 'output' to choose a name and a directory for those files
    'output': 'chains/my_euclid_experiment',
    #'debug': Cobaya's protected key of the input dictionary.
    # (UC): how much information you want Cobaya to print? If debug: True, it prints every single detail
    # that is going on internally in Cobaya
    'debug': False,
    #'timing': Cobaya's protected key of the input dictionary.
    # (UC): if timing: True, Cobaya returns how much time it took it to make a computation of the posterior
    # and how much time take each of the modules to perform their tasks
    'timing': True,
    #'force': Cobaya's protected key of the input dictionary.
    # (UC): if 'force': True, Cobaya forces deleting the previous output files, if found, with the same name
    'force': True}

In [ ]:
#'likelihood': Cobaya's protected key of the input dictionary.
# (UC): The user can select which data wants to use for the analysis.
# Check Cobaya's documentation to see the list of the current available data experiments
# In this DEMO, we load the Euclid-Likelihood as an external function, and name it 'Euclid'
info['likelihood'] = {'Euclid': 
                     {'external': EuclidLikelihood, # Likelihood Class to be read as external
                     # Note: everything down below will overwrite the information read
                     # in the config folder
                     #
                     # Select which observables want to use during the analysis
                     # by setting them to True or False
                     'observables_selection': {
                         'WL': {'WL': True, 'GCphot': True, 'GCspectro': False},
                         'GCphot': {'GCphot': True, 'GCspectro': False},
                         'GCspectro': {'GCspectro': True},
                         'CG': {'CG': False},
                         'add_phot_RSD': False,
                         'matrix_transform_phot': False
                     },
                     'use_magnification_bias_spectro': 0,
                     'use_Weyl': False,
                     # Plot the selected observables matrix
                     'plot_observables_selection': True, 
                     'solver': 'camb',
                      # Nonlinear flags
                      # With this, the user can specify which nonlinear model they want
                      # For the time-being the available options are: 
                      # NL_flag_phot_matter
                        # 0 -> linear-only
                        # 1 -> Takahashi
                        # 2 -> Mead2016 (w/o baryon corrections)
                        # 3 -> Mead2020 (w/o baryon corrections)
                        # 4 -> EE2
                        # 5 -> Bacco (matter)
                      # NL_flag_spectro
                        # 0 -> linear-only
                        # 1 -> EFT
                     'NL_flag_phot_matter': 3,
                     'NL_flag_spectro': 0,
                      # IA_flag
                        # 0 -> NLA
                        # 1 -> TATT
                     'IA_flag':0,
                     #
                      # Baryon flag
                      # With this, the user can specify which baryon model they want
                      # For the time-being the available options are: 
                            #0 -> No baryonic feedback
                            #1 -> Mead2016 (baryons)
                            #2 -> Mead2020_feedback
                            #3 -> BCemu baryons
                            #4 -> Bacco baryons
                     'NL_flag_phot_baryon': 0,
                     'NL_flag_phot_bias': 0,
                     'IR_resum': 'DST',
                     # This flag sets the redshift evolution for baryonic parameters for emulators
                     # The options are:
                            # True -> use X(z) = X_0 * (1+z)^(-nu_X), no. of params: 7*2 = 14
                            # False -> use X_i at each redshift bin i and interpolate, no. of params: 7*10 = 70
                     'Baryon_redshift_model': False,
                     'f_out_z_dep': False,
                     # Use modified gravity gamma
                     'use_gamma_MG': False,
                     # Add spectroscopic redshift errors
                     'GCsp_z_err' : True,
                     'k_max_extrap': 500.0,
                     'k_min_extrap': 1E-5,
                     'k_samp': 1000,
                     # z limit values and size z-array
                     'z_min': 0.0,
                     'z_max': 4.0,
                     'z_samp': 100,
                     'params': {
                    # (UC): each parameter below (which is a 'key' of another sub-dictionary) can contain a dictionary
                    # with the key 'prior', 'latex'...
                    # If the prior dictionary is not passed to a parameter, this parameter is fixed.
                    # In this example, we are sampling the parameter ns
                    # For more information see: https://cobaya.readthedocs.io/en/latest/example.html
                    'ombh2': 0.022445, #Omega density of baryons times the reduced Hubble parameter squared
                    'omch2': 0.1205579307, #Omega density of cold dark matter times the reduced Hubble parameter squared
                    'H0': 67, #Hubble parameter evaluated today (z=0) in km/s/Mpc
                    'tau': 0.0925, #optical depth
                    'mnu': 0.06, #  sum of the mass of neutrinos in eV
                    'nnu': 3.046, #N_eff of relativistic species 
                    'As': 2.12605e-9, #Amplitude of the primordial scalar power spectrum
                    'ns': 0.96, # primordial power spectrum tilt
                    'w': -1, #Dark energy fluid model
                    'wa': 0, #Dark energy fluid model
                    'omk': 0.0, #curvature density
                    #'omegam': None, #DERIVED parameter: Omega matter density
                    #'omegab': None, #DERIVED parameter: Omega baryon density
                    #'omeganu': None, #DERIVED parameter: Omega neutrino density
                    #'omnuh2': None, #DERIVED parameter: Omega neutrino density times de reduced Hubble parameter squared
                    #'omegac': None, #DERIVED parameter: Omega cold dark matter density
                    #'N_eff': None,
                    # (UC): baryonic feedback for built-in models (Mead2016/Mead2020_feedback; see next cell)
                    # These parameters are considered only when the corresponding nonlinear modelling option is selected:
                    # HMCode_A_baryon: 3.13 # default value for Mead2016
                    # HMCode_eta_baryon: 0.603 # default value for Mead2016
                    # HMCode_logT_AGN: 7.8 # default value for Mead2020_feedback
                    # (UC): galaxy bias parameters:
                    # The bias parameters below are currently fixed to the
                    # values used by the Inter Science Taskforce: Forecast (IST:F)
                    # and presented in the corresponding IST:F paper (arXiv: 1910.09273).
                    # However, they can be changed by the user and even sample over them by putting a prior
                    # Photometric bias parameters (defaults for 10 bins)
                    'b1_photo': 1.0997727037892875,
                    'b2_photo': 1.220245876862528,
                    'b3_photo': 1.2723993083933989,
                    'b4_photo': 1.316624471897739,
                    'b5_photo': 1.35812370570578,
                    'b6_photo': 1.3998214171814918,
                    'b7_photo': 1.4446452851824907,
                    'b8_photo': 1.4964959071110084,
                    'b9_photo': 1.5652475842498528,
                    'b10_photo': 1.7429859437184225,
                    # Adding these here just for generation of params.yaml
                    'b11_photo': 1.7429859437184225,
                    'b12_photo': 1.7429859437184225,
                    'b13_photo': 1.7429859437184225,
                    # These values are the defaults for 13 bins 
                    #'b1_photo': 1.03047,
                    #'b2_photo': 1.06699,
                    #'b3_photo': 1.17363,
                    #'b4_photo': 1.23340,
                    #'b5_photo': 1.27510,
                    #'b6_photo': 1.28574,
                    #'b7_photo': 1.39434,
                    #'b8_photo': 1.49170,
                    #'b9_photo': 1.52334,
                    #'b10_photo': 1.54639,
                    #'b11_photo': 1.68682,
                    #'b12_photo': 2.03066,
                    #'b13_photo': 2.57812,
                    # Bias parameters for polynomial model
                    'b0_poly_photo': 0.830703,
                    'b1_poly_photo': 1.190547,
                    'b2_poly_photo': -0.928357,
                    'b3_poly_photo': 0.423292,
                    # Magnification bias parameters
                    'magnification_bias_1': -0.8499,
                    'magnification_bias_2': -1.023,
                    'magnification_bias_3': -0.9474,
                    'magnification_bias_4': -0.9837,
                    'magnification_bias_5': -0.7699,
                    'magnification_bias_6': -0.7137,
                    'magnification_bias_7': -0.52345,
                    'magnification_bias_8': -0.40845,
                    'magnification_bias_9': -0.41045,
                    'magnification_bias_10': -0.0824,
                    'magnification_bias_11': 0.4917,
                    'magnification_bias_12': 0.98555,
                    'magnification_bias_13': 1.6056,
                    # Shear calibration multiplicative bias parameters                                                                                                                                                                                                                                                                                                            
                    'multiplicative_bias_1': 0.0,
                    'multiplicative_bias_2': 0.0,
                    'multiplicative_bias_3': 0.0,
                    'multiplicative_bias_4': 0.0,
                    'multiplicative_bias_5': 0.0,
                    'multiplicative_bias_6': 0.0,
                    'multiplicative_bias_7': 0.0,
                    'multiplicative_bias_8': 0.0,
                    'multiplicative_bias_9': 0.0,
                    'multiplicative_bias_10': 0.0,
                    'multiplicative_bias_11': 0.0,
                    'multiplicative_bias_12': 0.0,
                    'multiplicative_bias_13': 0.0,
                    ############################
                    # Spectroscopic parameters #
                    ############################
                    # Linear bias #
                    'b1_spectro_bin1': 1.46, 'b1_spectro_bin2': 1.61, 'b1_spectro_bin3': 1.75, 'b1_spectro_bin4': 1.90,
                    # Local quadratic bias #
                    'b2_spectro_bin1': 0.0, 'b2_spectro_bin2': 0.0, 'b2_spectro_bin3': 0.0, 'b2_spectro_bin4': 0.0,
                    # FOG counterterms #
                    'c0_spectro_bin1': 0.0, 'c0_spectro_bin2': 0.0, 'c0_spectro_bin3': 0.0, 'c0_spectro_bin4': 0.0,
                    'c2_spectro_bin1': 0.0, 'c2_spectro_bin2': 0.0, 'c2_spectro_bin3': 0.0, 'c2_spectro_bin4': 0.0,
                    'c4_spectro_bin1': 0.0, 'c4_spectro_bin2': 0.0, 'c4_spectro_bin3': 0.0, 'c4_spectro_bin4': 0.0,
                    # Deviations from Poisson shot-noise #
                    'aP_spectro_bin1': 0.0, 'aP_spectro_bin2': 0.0, 'aP_spectro_bin3': 0.0, 'aP_spectro_bin4': 0.0,
                    # Inverse number density #
                    'Psn_spectro_bin1': 0.0, 'Psn_spectro_bin2': 0.0, 'Psn_spectro_bin3': 0.0, 'Psn_spectro_bin4': 0.0,
                    # Intrinsic alignment parameters
                    'a1_ia': 1.72,
                    'a2_ia': 2,
                    'b1_ia': 1,
                    'eta1_ia': -0.41,
                    'eta2_ia': 1,
                    'beta1_ia': 0.0,
                    'pivot_redshift': 0.0,
                    # Redshift distributions nuisance parameters: shifts
                    'dz_1_GCphot': 0., 'dz_1_WL': 0.,
                    'dz_2_GCphot': 0., 'dz_2_WL': 0.,
                    'dz_3_GCphot': 0., 'dz_3_WL': 0.,
                    'dz_4_GCphot': 0., 'dz_4_WL': 0.,
                    'dz_5_GCphot': 0., 'dz_5_WL': 0.,
                    'dz_6_GCphot': 0., 'dz_6_WL': 0.,
                    'dz_7_GCphot': 0., 'dz_7_WL': 0.,
                    'dz_8_GCphot': 0., 'dz_8_WL': 0.,
                    'dz_9_GCphot': 0., 'dz_9_WL': 0.,
                    'dz_10_GCphot': 0., 'dz_10_WL': 0.,
                    'dz_11_GCphot': 0., 'dz_11_WL': 0.,
                    'dz_12_GCphot': 0., 'dz_12_WL': 0.,
                    'dz_13_GCphot': 0., 'dz_13_WL': 0.,
                    ###############################################################################
                    # Baryonic parameters
                    # Two emulators have different parameters and different ways of setting them:
                    #  - Parameters can be set for each redshift bin as par_i, with i= 1, ..., 13  (currently max 13 bins)
                    #  - Or as a power law in redshift par(z) = par_0 * (1+z)^(-nu_par)
                    ###############################################################################
                    # Example BCemu baryon params (from fit to BAHAMAS for 10 bins, values for bins 11 - 13 are the same as bin 10)
                    'log10Mc_bcemu_bin1': 13.402342835406102, 'log10Mc_bcemu_bin2': 13.255603572838046, 'log10Mc_bcemu_bin3': 13.39534479993441,
                    'log10Mc_bcemu_bin4': 13.491189798802624, 'log10Mc_bcemu_bin5': 13.517099731729203, 'log10Mc_bcemu_bin6': 13.431948448938623,
                    'log10Mc_bcemu_bin7': 13.159142065726643, 'log10Mc_bcemu_bin8': 12.630530198678242, 'log10Mc_bcemu_bin9': 11.768928839012311,
                    'log10Mc_bcemu_bin10': 11.123969034839764,'log10Mc_bcemu_bin11': 11.123969034839764,'log10Mc_bcemu_bin12': 11.123969034839764,
                    'log10Mc_bcemu_bin13': 11.123969034839764,
                    'mu_bcemu_bin1': 1.081169575215787, 'mu_bcemu_bin2': 1.0748796706532235, 'mu_bcemu_bin3': 1.257505674386812, 
                    'mu_bcemu_bin4': 1.3953104854720346, 'mu_bcemu_bin5': 1.4707848100185459, 'mu_bcemu_bin6': 1.451326233210004, 
                    'mu_bcemu_bin7': 1.2738670043792995, 'mu_bcemu_bin8': 0.8851923901330802, 'mu_bcemu_bin9': 0.23572246186875243, 
                    'mu_bcemu_bin10': -0.006953332782079408, 'mu_bcemu_bin11': -0.006953332782079408, 'mu_bcemu_bin12': -0.006953332782079408,
                    'mu_bcemu_bin13': -0.006953332782079408, 
                    'thej_bcemu_bin1': 3.6711008267835776, 'thej_bcemu_bin2': 4.0853297524846175, 'thej_bcemu_bin3': 4.1001870114090835,
                    'thej_bcemu_bin4': 4.085897535238355, 'thej_bcemu_bin5': 4.062128106370927, 'thej_bcemu_bin6': 4.0416849190138695, 
                    'thej_bcemu_bin7': 4.0405617133891685, 'thej_bcemu_bin8': 4.073797196972992, 'thej_bcemu_bin9': 4.168188042786498, 
                    'thej_bcemu_bin10': 4.567495847826163, 'thej_bcemu_bin11': 4.567495847826163, 'thej_bcemu_bin12': 4.567495847826163, 
                    'thej_bcemu_bin13': 4.567495847826163,
                    'gamma_bcemu_bin1': 2.732738146480909, 'gamma_bcemu_bin2': 2.794658925091356, 'gamma_bcemu_bin3': 3.008141783989797,
                    'gamma_bcemu_bin4': 3.2186226058208787, 'gamma_bcemu_bin5': 3.425034377127418, 'gamma_bcemu_bin6': 3.6237277892257937, 
                    'gamma_bcemu_bin7': 3.805233587893605, 'gamma_bcemu_bin8': 3.952453083293017, 'gamma_bcemu_bin9': 4.012632091622541,
                    'gamma_bcemu_bin10': 3.1454520255604006, 'gamma_bcemu_bin11': 3.1454520255604006, 'gamma_bcemu_bin12': 3.1454520255604006,
                    'gamma_bcemu_bin13': 3.1454520255604006,
                    'delta_bcemu_bin1': 6.407442578598766, 'delta_bcemu_bin2': 7.0846518716943025, 'delta_bcemu_bin3': 7.226700228338714,
                    'delta_bcemu_bin4': 7.343194626402527, 'delta_bcemu_bin5': 7.470370166898234, 'delta_bcemu_bin6': 7.638743359373677,
                    'delta_bcemu_bin7': 7.891612514615537, 'delta_bcemu_bin8': 8.252389998618296, 'delta_bcemu_bin9': 8.702184867164187,
                    'delta_bcemu_bin10': 7.994110032082859, 'delta_bcemu_bin11': 7.994110032082859, 'delta_bcemu_bin12': 7.994110032082859,
                    'delta_bcemu_bin13': 7.994110032082859,
                    'eta_bcemu_bin1': 0.20680675958346154, 'eta_bcemu_bin2': 0.22813237994091348, 'eta_bcemu_bin3': 0.21989900653826788,
                    'eta_bcemu_bin4': 0.2090499814223423, 'eta_bcemu_bin5': 0.19645554090162448, 'eta_bcemu_bin6': 0.18238365514915927, 
                    'eta_bcemu_bin7': 0.16686344766458094, 'eta_bcemu_bin8': 0.14883503959461816, 'eta_bcemu_bin9': 0.1240089652119986,
                    'eta_bcemu_bin10': 0.044931761090432315, 'eta_bcemu_bin11': 0.044931761090432315, 'eta_bcemu_bin12': 0.044931761090432315,
                    'eta_bcemu_bin13': 0.044931761090432315,
                    'deta_bcemu_bin1': 0.08189803795251066, 'deta_bcemu_bin2': 0.05028436917683151, 'deta_bcemu_bin3': 0.04842000359519622,
                    'deta_bcemu_bin4': 0.04869423933122641, 'deta_bcemu_bin5': 0.04959595282843388, 'deta_bcemu_bin6': 0.05011878848954406,
                    'deta_bcemu_bin7': 0.04907477885822071, 'deta_bcemu_bin8': 0.04681178740273659, 'deta_bcemu_bin9': 0.048062862071344824,
                    'deta_bcemu_bin10': 0.14550946518064195, 'deta_bcemu_bin11': 0.14550946518064195, 'deta_bcemu_bin12': 0.14550946518064195,
                    'deta_bcemu_bin13': 0.14550946518064195,
                    # BCemu params for the redshift parametrization
                    # To mimic the BCemu 3-parameter model, just vary the first 6 params and leave the others fixed to these values.
                    # A 1-parameter model would use just log10Mc (and others fixed to current values)
                    'log10Mc_bcemu_0': 13.32,
                    'nu_log10Mc_bcemu': -0.15,
                    'thej_bcemu_0': 3.5,
                    'nu_thej_bcemu': 0.0,
                    'deta_bcemu_0': 0.2,
                    'nu_deta_bcemu': 0.6,
                    'mu_bcemu_0': 1.0,
                    'nu_mu_bcemu': 0.0,
                    'gamma_bcemu_0': 2.5,
                    'nu_gamma_bcemu': 0.0,
                    'delta_bcemu_0': 7.0,
                    'nu_delta_bcemu': 0.0,
                    'eta_bcemu_0': 0.2,
                    'nu_eta_bcemu': 0.0,
                    # Example Bacco baryon parameters (from fit to BAHAMAS for 10 bins, values for bins 11 - 13 are the same as bin 10)
                    'M_c_bacco_bin1': 14.158079090897681,'M_c_bacco_bin2': 14.413536974582364,
                    'M_c_bacco_bin3': 14.513655381066439,
                    'M_c_bacco_bin4': 14.59501140993621,
                    'M_c_bacco_bin5': 14.667602932620955,
                    'M_c_bacco_bin6': 14.736117785720888,
                    'M_c_bacco_bin7': 14.803902874910724,
                    'M_c_bacco_bin8': 14.873354165369339,
                    'M_c_bacco_bin9': 14.947678648592403,
                    'M_c_bacco_bin10': 15.179943239223197,
                    'M_c_bacco_bin11': 15.179943239223197,
                    'M_c_bacco_bin12': 15.179943239223197,
                    'M_c_bacco_bin13': 15.179943239223197,
                    'eta_bacco_bin1': -0.34397323435383403, 'eta_bacco_bin2': -0.3823001659654948,
                    'eta_bacco_bin3': -0.3666759403458338,
                    'eta_bacco_bin4': -0.34766555599217897,
                    'eta_bacco_bin5': -0.32723937222645033,
                    'eta_bacco_bin6': -0.3066851485202732,
                    'eta_bacco_bin7': -0.28793191687619096,
                    'eta_bacco_bin8': -0.2761849439738153,
                    'eta_bacco_bin9': -0.29044476508897765,
                    'eta_bacco_bin10': -0.3012527587604943,
                    'eta_bacco_bin11': -0.3012527587604943,
                    'eta_bacco_bin12': -0.3012527587604943,
                    'eta_bacco_bin13': -0.3012527587604943,
                    'beta_bacco_bin1': -0.1401597529600361,'beta_bacco_bin2': -0.14602731261184787,
                    'beta_bacco_bin3': -0.15945179021822595,
                    'beta_bacco_bin4': -0.1717165402260191,
                    'beta_bacco_bin5': -0.18260340867450087,
                    'beta_bacco_bin6': -0.1914516575772147,
                    'beta_bacco_bin7': -0.19665359534532956,
                    'beta_bacco_bin8': -0.1938633002915493,
                    'beta_bacco_bin9': -0.16799956045034847,
                    'beta_bacco_bin10': -0.1180328507054961,
                    'beta_bacco_bin11': -0.1180328507054961,
                    'beta_bacco_bin12': -0.1180328507054961,
                    'beta_bacco_bin13': -0.1180328507054961,
                    'M1_z0_cen_bacco_bin1': 11.340025775544985,'M1_z0_cen_bacco_bin2': 11.30961906881156,
                    'M1_z0_cen_bacco_bin3': 11.366002025818862,
                    'M1_z0_cen_bacco_bin4': 11.423530231429538,
                    'M1_z0_cen_bacco_bin5': 11.479266623369345,
                    'M1_z0_cen_bacco_bin6': 11.529947744472405,
                    'M1_z0_cen_bacco_bin7': 11.569052829475833,
                    'M1_z0_cen_bacco_bin8': 11.578833725581436,
                    'M1_z0_cen_bacco_bin9': 11.496367669453079,
                    'M1_z0_cen_bacco_bin10': 11.360811343412026,
                    'M1_z0_cen_bacco_bin11': 11.360811343412026,
                    'M1_z0_cen_bacco_bin12': 11.360811343412026,
                    'M1_z0_cen_bacco_bin13': 11.360811343412026,
                    'theta_inn_bacco_bin1': -0.9790217925155928, 'theta_inn_bacco_bin2': -1.3854647301685985,
                    'theta_inn_bacco_bin3': -1.3604452515853098,
                    'theta_inn_bacco_bin4': -1.2949667258085438,
                    'theta_inn_bacco_bin5': -1.2056425079533284,
                    'theta_inn_bacco_bin6': -1.0991845011332575,
                    'theta_inn_bacco_bin7': -0.9805620838281162,
                    'theta_inn_bacco_bin8': -0.8624918408856748,
                    'theta_inn_bacco_bin9': -0.7968588249755786,
                    'theta_inn_bacco_bin10': -0.5312787510657424,
                    'theta_inn_bacco_bin11': -0.5312787510657424,
                    'theta_inn_bacco_bin12': -0.5312787510657424,
                    'theta_inn_bacco_bin13': -0.5312787510657424,
                    'M_inn_bacco_bin1': 12.472,'M_inn_bacco_bin2': 11.098133175636903,
                    'M_inn_bacco_bin3': 10.685545014634902,
                    'M_inn_bacco_bin4': 10.379401082211185,
                    'M_inn_bacco_bin5': 10.124549632511826,
                    'M_inn_bacco_bin6': 9.89485323082124,
                    'M_inn_bacco_bin7': 9.669674533440968,
                    'M_inn_bacco_bin8': 9.423522683043274,
                    'M_inn_bacco_bin9': 9.086255671995564,
                    'M_inn_bacco_bin10': 8.119132619254136,
                    'M_inn_bacco_bin11': 8.119132619254136,
                    'M_inn_bacco_bin12': 8.119132619254136,
                    'M_inn_bacco_bin13': 8.119132619254136,
                    'theta_out_bacco_bin1': 0.2672123790126238,'theta_out_bacco_bin2': 0.21845633941784046,
                    'theta_out_bacco_bin3': 0.22229391886311567,
                    'theta_out_bacco_bin4': 0.23124624733764712,
                    'theta_out_bacco_bin5': 0.24338032280596764,
                    'theta_out_bacco_bin6': 0.25802109222979985,
                    'theta_out_bacco_bin7': 0.27483978056155306,
                    'theta_out_bacco_bin8': 0.29287229590732955,
                    'theta_out_bacco_bin9': 0.3076030069491944,
                    'theta_out_bacco_bin10': 0.3569278707998519,
                    'theta_out_bacco_bin11': 0.3569278707998519,
                    'theta_out_bacco_bin12': 0.3569278707998519,
                    'theta_out_bacco_bin13': 0.3569278707998519,
                    # Bacco params for the redshift parametrization
                    # Here too, one can use just the first 6 params to get a simpler model, with the others fixed.
                    # Note that the redshift dependence has not been tested by the Bacco collaboration.
                    'M_c_bacco_0': 13.25,
                    'nu_M_c_bacco': -0.15,
                    'eta_bacco_0': -0.3,
                    'nu_eta_bacco': 0.0,
                    'beta_bacco_0': -0.22,
                    'nu_beta_bacco': 0.0,
                    'M1_z0_cen_bacco_0': 10.5,
                    'nu_M1_z0_cen_bacco': 0.0,
                    'theta_inn_bacco_0': -0.86,
                    'nu_theta_inn_bacco': 0.0,
                    'M_inn_bacco_0': 13.4,
                    'nu_M_inn_bacco': 0.0,
                    'theta_out_bacco_0': 0.25,
                    'nu_theta_out_bacco': 0.0,
                    # Growth index
                    'gamma_MG': 0.55,
                    # Purity 
                    'f_out':0.0,
                    'f_out_1':0.0,
                    'f_out_2': 0.0,
                    'f_out_3': 0.0,
                    'f_out_4': 0.0,
                    'sigma_z': 0.002},
                     #'data': This give specifications for the paths of the input data files
                     'data': { 
                        #'sample' specifies the first folder below the main data folder
                        'sample': 'ExternalBenchmark',
                        #'spectro' and 'photo' specify paths to data files.
                        'spectro': {
                            # GC Spectro root name should contain z{:s} string
                            # to enable iteration over bins
                            'root': 'cov_power_galaxies_dk0p004_z{:s}.fits',
                            'redshifts': ["1.", "1.2", "1.4", "1.65"],
                            'edges': [0.9, 1.1, 1.3, 1.5, 1.8],
                            'Fourier': True,
                            'root_mixing_matrix': 'mm_FS230degCircle_m3_nosm_obsz_z0.9-1.1.fits',
                            'scale_cuts_fourier': 'GCspectro-FourierSpace.yaml'},
                        'photo': {
                            'redshifts': [0.2095, 0.489, 0.619, 0.7335, 0.8445, 0.9595, 1.087, 1.2395, 1.45, 2.038],
                            'ndens_GC': 'niTab-EP10-RB00.dat',
                            'ndens_WL': 'niTab-EP10-RB00.dat',
                            'luminosity_ratio': 'luminosity_ratio.dat',
                            'Fourier': True,
                            # Photometric root names should contain z{:s} string
                            # to specify IA model
                            'root_GC': 'Cls_{:s}_PosPos.dat',
                            'root_WL': 'Cls_{:s}_ShearShear.dat',
                            'root_XC': 'Cls_{:s}_PosShear.dat',
                            'IA_model': 'zNLA',
                            'Fourier': True,
                            'photo_data': 'standard',
                            'root_mixing_matrix': 'fs2_mms_10zbins_32ellbins.fits',
                            # Photometric covariances root names should contain z{:s} string
                            # to specify how the covariance was calculated
                            'cov_GC': 'CovMat-PosPos-{:s}-20Bins.npz',
                            'cov_WL': 'CovMat-ShearShear-{:s}-20Bins.npz',
                            'cov_3x2pt': 'CovMat-3x2pt-{:s}-20Bins.npz',
                            'cov_model': 'Gauss'}},
                   
                    }}

In [ ]:
from cobaya.model import get_model

from cloe.auxiliary.likelihood_yaml_handler import set_halofit_version
set_halofit_version(info, info['likelihood']['Euclid']['NL_flag_phot_matter'], 
                          info['likelihood']['Euclid']['NL_flag_phot_baryon'])
# The `get_model` function of Cobaya imported in the line above needs a yaml or dictionary as an argument
# 
# We measure the time to give us an estimation of how much time it takes to make the initialization of the
# likelihood
t1 = time.time()

# Second: create an instance of the `model` wrapper called model
model = get_model(info)
print('Time for initialization of the likelihood: ', time.time()-t1)
# We need to sample the log posterior to create model.provider.params
logposterior = model.logposterior({})

In [ ]:
# Create an instance of the class EuclidLikelihood and initialize it (general case)
like = EuclidLikelihood()
like.initialize()
like.passing_requirements(model, info, **model.provider.params)


In [ ]:
# New model with halofit
info['likelihood']['Euclid']['NL_flag_phot_matter']=1
info['likelihood']['Euclid']['NL_flag_phot_baryon']=0
set_halofit_version(info, info['likelihood']['Euclid']['NL_flag_phot_matter'],
                        info['likelihood']['Euclid']['NL_flag_phot_baryon'])
model_hf = get_model(info)
logposterior = model_hf.logposterior({})

In [ ]:
# New instances of the class EuclidLikelihood with halofit
like_hf = EuclidLikelihood()
like_hf.initialize()
like_hf.passing_requirements(model_hf, info, **model_hf.provider.params)

In [ ]:
# New model with Mead2016
info['likelihood']['Euclid']['NL_flag_phot_matter']=2
info['likelihood']['Euclid']['NL_flag_phot_baryon']=0
set_halofit_version(info, info['likelihood']['Euclid']['NL_flag_phot_matter'],
                        info['likelihood']['Euclid']['NL_flag_phot_baryon'])
model_mead16 = get_model(info)
logposterior = model_mead16.logposterior({})

In [ ]:
# New instances of the class EuclidLikelihood with Mead2016
like_mead16 = EuclidLikelihood()
like_mead16.initialize()
like_mead16.passing_requirements(model_mead16, info, **model_mead16.provider.params)

In [ ]:
# New model with Mead2016
info_16f = copy.deepcopy(info)
info_16f['likelihood']['Euclid']['NL_flag_phot_matter']=2
info_16f['likelihood']['Euclid']['NL_flag_phot_baryon']=1
info_16f['params']['HMCode_A_baryon'] =  3.01 # default value for Mead2016 owls_dblim
info_16f['params']['HMCode_eta_baryon'] = 0.70

set_halofit_version(info_16f, info_16f['likelihood']['Euclid']['NL_flag_phot_matter'],
                        info_16f['likelihood']['Euclid']['NL_flag_phot_baryon'])
model_mead16f = get_model(info_16f)
logposterior = model_mead16f.logposterior({})

In [ ]:
# New instances of the class EuclidLikelihood with Mead2016
like_mead16f = EuclidLikelihood()
like_mead16f.initialize()
like_mead16f.passing_requirements(model_mead16f, info_16f, **model_mead16f.provider.params)

In [ ]:
# New model with Mead2020_feedback
info_20f=copy.deepcopy(info)
info_20f['likelihood']['Euclid']['NL_flag_phot_matter']=3
info_20f['likelihood']['Euclid']['NL_flag_phot_baryon']=2
info_20f['params']['HMCode_logT_AGN'] = 7.8 # default value for Mead2020_feedback
set_halofit_version(info_20f, info_20f['likelihood']['Euclid']['NL_flag_phot_matter'],
                        info_20f['likelihood']['Euclid']['NL_flag_phot_baryon'])
model_mead20fb = get_model(info_20f)
logposterior = model_mead20fb.logposterior({})

In [ ]:
# New instances of the class EuclidLikelihood with Mead2020_feedback
like_mead20fb = EuclidLikelihood()
like_mead20fb.initialize()
like_mead20fb.passing_requirements(model_mead20fb, info_20f, **model_mead20fb.provider.params)

### Copy the cosmo dictionary and produce new ones for emulators

In [ ]:
# This is just the scale at which to get the growth factor, it is used as an input for update_cosmo_dic
wavenumber_growth = 0.05

# The cases with NL_flag_phot_matter=1,2,4 are special, because they need the different initializations of like and model
like_hf.cosmo.cosmo_dic['NL_flag_phot_matter'] = 1
like_hf.cosmo.update_cosmo_dic(like_hf.cosmo.cosmo_dic['z_win'], wavenumber_growth)
cosmo_dic_1 = copy.copy(like_hf.cosmo.cosmo_dic)

like_mead16.cosmo.cosmo_dic['NL_flag_phot_matter'] = 2
like_mead16.cosmo.update_cosmo_dic(like_mead16.cosmo.cosmo_dic['z_win'], wavenumber_growth)
cosmo_dic_2 = copy.copy(like_mead16.cosmo.cosmo_dic)

# These can all be done this way because they all use hmcode (when needed)
like.cosmo.cosmo_dic['NL_flag_phot_matter'] = 0
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], wavenumber_growth)
cosmo_dic_0 = copy.copy(like.cosmo.cosmo_dic)

like.cosmo.cosmo_dic['NL_flag_phot_matter'] = 3
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], wavenumber_growth)
cosmo_dic_3 = copy.copy(like.cosmo.cosmo_dic)

like.cosmo.cosmo_dic['NL_flag_phot_matter'] = 4
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], wavenumber_growth)
cosmo_dic_4 = copy.copy(like.cosmo.cosmo_dic)

like.cosmo.cosmo_dic['NL_flag_phot_matter'] = 5
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], wavenumber_growth)
cosmo_dic_5 = copy.copy(like.cosmo.cosmo_dic)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## Choose wavenumber-array for plots
wavenumbers=np.logspace(-3, 1, 200)

In [ ]:
## Obtain set of defined redshift range
like.cosmo.cosmo_dic['z_win']

In [ ]:
## Choose a set of z's to plot

redshift_chosen = [like.cosmo.cosmo_dic['z_win'][ii] for ii in [0,10,37,50]]
redshift_chosen

### Plot the Pk_delta for all models

Pk_delta is just the linear camb Pk in all cases. Plotting it as a null test.

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(12,9))
fig.suptitle('Linear Spectra')

ax=axs

redshift0=redshift_chosen[0]

ax.loglog(wavenumbers, cosmo_dic_0['Pk_delta'].P(redshift0, wavenumbers), '.-', 
           label=r"NL_flag_phot_matter = 0")
ax.set_xlabel(r'$k$ $\left[{\rm Mpc}^{-1}\right]$')
ax.set_ylabel(r'$P_m$ $\left[{\rm Mpc}^{-3}\right]$')
ax.legend(frameon=False)

ax.loglog(wavenumbers, cosmo_dic_1['Pk_delta'].P(redshift0, wavenumbers), '.-', 
           label=r"NL_flag_phot_matter = 1")
ax.set_xlabel(r'$k$ $\left[{\rm Mpc}^{-1}\right]$')
ax.set_ylabel(r'$P_m$ $\left[{\rm Mpc}^{-3}\right]$')
ax.legend(frameon=False)

ax.loglog(wavenumbers, cosmo_dic_2['Pk_delta'].P(redshift0, wavenumbers), '.-', 
           label=r"NL_flag_phot_matter = 2")
ax.set_xlabel(r'$k$ $\left[{\rm Mpc}^{-1}\right]$')
ax.set_ylabel(r'$P_m$ $\left[{\rm Mpc}^{-3}\right]$')
ax.legend(frameon=False)

ax.loglog(wavenumbers, cosmo_dic_3['Pk_delta'].P(redshift0, wavenumbers), '.-', 
           label=r"NL_flag_phot_matter = 3")
ax.set_xlabel(r'$k$ $\left[{\rm Mpc}^{-1}\right]$')
ax.set_ylabel(r'$P_m$ $\left[{\rm Mpc}^{-3}\right]$')
ax.legend(frameon=False)

ax.loglog(wavenumbers, cosmo_dic_4['Pk_delta'].P(redshift0, wavenumbers), '.-', 
           label=r"NL_flag_phot_matter = 4")
ax.set_xlabel(r'$k$ $\left[{\rm Mpc}^{-1}\right]$')
ax.set_ylabel(r'$P_m$ $\left[{\rm Mpc}^{-3}\right]$')
ax.legend(frameon=False)

ax.loglog(wavenumbers, cosmo_dic_5['Pk_delta'].P(redshift0, wavenumbers), '.-', 
           label=r"NL_flag_phot_matter = 5")
ax.set_xlabel(r'$k$ $\left[{\rm Mpc}^{-1}\right]$')
ax.set_ylabel(r'$P_m$ $\left[{\rm Mpc}^{-3}\right]$')
ax.legend(frameon=False)

In [ ]:
from matplotlib.pyplot import cm
from matplotlib.cm import get_cmap

In [ ]:
## Define a color palette for the plots
name = "Dark2"
cmap = get_cmap(name)  # type: matplotlib.colors.ListedColormap
colors = cmap.colors

### Plot the Pgg_photo

In [ ]:
## Create four panels, one for each of the chosen redshifts above

fig, axs = plt.subplots(2, 2, figsize=(24,18))
fig.suptitle('Spectra')

ax=axs[0,0]

Pab = 'Pgg_phot'


for iz in range(4):
    if iz==0:
        ax=axs[0,0]
    if iz==1:
        ax=axs[0,1]
    if iz==2:
        ax=axs[1,0]
    if iz==3:
        ax=axs[1,1]
    redshift0=redshift_chosen[iz]

    ax.loglog(wavenumbers, cosmo_dic_0['Pk_delta'].P(redshift0, wavenumbers), ':', 
               label=r"Linear CAMB Pk z={:.2f}".format(redshift0), c=colors[0])
    
    
    ax.loglog(wavenumbers, cosmo_dic_0[Pab](redshift0, wavenumbers).flatten(), '--', 
               label=r"Linear recipe z={:.2f}".format(redshift0), c=colors[1])


    ax.loglog(wavenumbers, cosmo_dic_1[Pab](redshift0, wavenumbers).flatten(), '--', 
               label=r"Halofit z={:.2f}".format(redshift0), c=colors[2])
    
    
    ax.loglog(wavenumbers, cosmo_dic_2[Pab](redshift0, wavenumbers).flatten(), '--', 
               label=r"Mead2016 z={:.2f}".format(redshift0), c=colors[6])

    
    ax.loglog(wavenumbers, cosmo_dic_3[Pab](redshift0, wavenumbers).flatten(), '--', 
               label=r"Mead2020 z={:.2f}".format(redshift0), c=colors[3])
    

    ax.loglog(wavenumbers, cosmo_dic_4[Pab](redshift0, wavenumbers).flatten(), '.-', 
               label=r"Euclid Emulator z={:.2f}".format(redshift0), c=colors[4])


    ax.loglog(wavenumbers, cosmo_dic_5[Pab](redshift0, wavenumbers).flatten(), '.-', 
               label=r"BACCO Emulator z={:.2f}".format(redshift0), c=colors[5])


    ax.set_xlabel(r'$k$ $[{Mpc}^{-1}]$', fontsize=24)
    ax.set_ylabel(r'$P_{gg}^{phot} [{Mpc}^{-3}]$')
    ax.legend(frameon=False, fontsize=24)

In [ ]:
## Now the ratios

fig, axs = plt.subplots(2, 2, figsize=(24,18))
fig.suptitle('Ratios to halofit')

ax=axs[0,0]

Pab = 'Pmm_phot'


for iz in range(4):
    if iz==0:
        ax=axs[0,0]
    if iz==1:
        ax=axs[0,1]
    if iz==2:
        ax=axs[1,0]
    if iz==3:
        ax=axs[1,1]
    redshift0=redshift_chosen[iz]
    
    #---
    
    ax.semilogx(wavenumbers, cosmo_dic_2[Pab](redshift0, wavenumbers).flatten()/cosmo_dic_1[Pab](redshift0, wavenumbers).flatten(), '--', 
               label=r"Mead2016 z={:.2f}".format(redshift0), c=colors[6])
    
    ax.semilogx(wavenumbers, cosmo_dic_3[Pab](redshift0, wavenumbers).flatten()/cosmo_dic_1[Pab](redshift0, wavenumbers).flatten(), '--', 
               label=r"Mead2020 z={:.2f}".format(redshift0), c=colors[3])
    #
    ax.semilogx(wavenumbers, cosmo_dic_4[Pab](redshift0, wavenumbers).flatten()/cosmo_dic_1[Pab](redshift0, wavenumbers).flatten(), '.-', 
               label=r"Euclid Emulator z={:.2f}".format(redshift0), c=colors[4])
    
    ax.semilogx(wavenumbers, cosmo_dic_5[Pab](redshift0, wavenumbers).flatten()/cosmo_dic_1[Pab](redshift0, wavenumbers).flatten(), '.-', 
               label=r"BACCO Emulator z={:.2f}".format(redshift0), c=colors[5])
    
    ax.set_xlabel(r'$k$ $[{Mpc}^{-1}]$', fontsize=24)
    ax.set_ylabel(r'$P_{mm}/P_{mm}^{\rm halofit} [{Mpc}^{-3}]$')
    
    ax.set_ylim([0.9,1.1])
    
    ax.legend(frameon=False, fontsize=24)

### Plotting the C_ells manually using the Photo class and data Reader

In [ ]:
from cloe.photometric_survey.photo import Photo
from cloe.data_reader.reader import Reader

In [ ]:
data = info['likelihood']['Euclid']['data']
read_data = Reader(data)
read_data.compute_nz()
read_data.read_phot()
# As we currently don't have a way to load fiducial r(z), d(z) and
# H(z)s, these are currently set to match the ones from the cosmo_dic.
# This will need to be fixed once this is decided.

In [ ]:
phot_ins_0 = Photo( cosmo_dic_0, read_data.nz_dict_WL,
                      read_data.nz_dict_GC_Phot)
phot_ins_1 = Photo( cosmo_dic_1, read_data.nz_dict_WL,
                      read_data.nz_dict_GC_Phot)
phot_ins_2 = Photo( cosmo_dic_2, read_data.nz_dict_WL,
                      read_data.nz_dict_GC_Phot)
phot_ins_3 = Photo( cosmo_dic_3, read_data.nz_dict_WL,
                      read_data.nz_dict_GC_Phot)
phot_ins_4 = Photo( cosmo_dic_4, read_data.nz_dict_WL,
                      read_data.nz_dict_GC_Phot)
phot_ins_5 = Photo( cosmo_dic_5, read_data.nz_dict_WL,
                      read_data.nz_dict_GC_Phot)

In [ ]:
ells= np.logspace(np.log10(10), np.log10(5000), 100)

In [ ]:
ClGC_0 = phot_ins_0.Cl_GC_phot
ClGC_1 = phot_ins_1.Cl_GC_phot
ClGC_2 = phot_ins_2.Cl_GC_phot
ClGC_3 = phot_ins_3.Cl_GC_phot
ClGC_4 = phot_ins_4.Cl_GC_phot
ClGC_5 = phot_ins_5.Cl_GC_phot

In [ ]:
ClWL_0 = phot_ins_0.Cl_WL
ClWL_1 = phot_ins_1.Cl_WL
ClWL_2 = phot_ins_2.Cl_WL
ClWL_3 = phot_ins_3.Cl_WL
ClWL_4 = phot_ins_4.Cl_WL
ClWL_5 = phot_ins_5.Cl_WL

In [ ]:
ClGCxWL_0 = phot_ins_0.Cl_cross
ClGCxWL_1 = phot_ins_1.Cl_cross
ClGCxWL_2 = phot_ins_2.Cl_cross
ClGCxWL_3 = phot_ins_3.Cl_cross
ClGCxWL_4 = phot_ins_4.Cl_cross
ClGCxWL_5 = phot_ins_5.Cl_cross

In [ ]:
def get_error_bars(bin_m, bin_n, no_bins=10, probe="GC-Phot"):
    nbins_wl = read_data.num_bins_wl
    nbins_xc = read_data.num_bins_xcphot
    nbins_gc = read_data.num_bins_gcphot
    nells_wl = read_data.num_scales_wl
    nells_xc = read_data.num_scales_xcphot
    nells_gc = read_data.num_scales_gcphot
    if bin_m <= bin_n:
            bin_i = bin_m
            bin_j = bin_n
    else:
            bin_i = bin_n
            bin_j = bin_m

    cov_diags = np.sqrt(np.diagonal(read_data.data_dict['3x2pt_cov']))
    counter = 0
    for i in range(1, no_bins + 1):
        for j in range(i, no_bins + 1):
            if i == bin_i and j == bin_j:
                cur_index = counter
            counter += 1
    
    if probe == 'WL':
        i_range = [cur_index + nbins_wl * cur_ell for cur_ell in range(nells_wl)]
    else:
        i_range = [nbins_wl * nells_wl + nbins_xc * nells_xc + \
            cur_index + nbins_gc * cur_ell for cur_ell in range(nells_gc)]
    
    return read_data.data_dict[probe]['ells'], cov_diags[i_range]

In [ ]:
def get_error_bars_XC(bin_WL, bin_GC, no_bins=10, probe='XC-Phot'):
    nbins_wl = read_data.num_bins_wl
    nbins_xc = read_data.num_bins_xcphot
    nells_wl = read_data.num_scales_wl
    nells_xc = read_data.num_scales_xcphot
    nells_gc = read_data.num_scales_gcphot

    cov_diags = np.sqrt(np.diagonal(read_data.data_dict['3x2pt_cov']))
    counter = 0
    for i in range(1, no_bins + 1):
        for j in range(1, no_bins + 1):
            if i == bin_WL and j == bin_GC:
                cur_index = counter
            counter += 1
    
    i_range = [nbins_wl * nells_wl + \
            cur_index + nbins_xc * cur_ell for cur_ell in range(nells_xc)]
        
    return read_data.data_dict[probe]['ells'], cov_diags[i_range]

### Plotting the WL C_ells

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(24,18), dpi=200)
fig.suptitle(r'WL $C^{ij}_{\ell}$')

iz_dict={1:[0,0], 3:[0,1], 5:[1,0], 10:[1,1]}

for iz in [1,3,5,10]:
    
    ax1 = axs[iz_dict[iz][0],iz_dict[iz][1]]
    
    pl_label_0 = 'Linear Recipe'
    ax1.loglog(ells, [ClWL_0(cur_ell, iz, iz) for cur_ell in ells], color=colors[1], ls='-.', label=pl_label_0, lw=3)
    
    pl_label_1 = 'Halofit'
    ax1.loglog(ells, [ClWL_1(cur_ell, iz, iz) for cur_ell in ells], color=colors[2], ls='--', label=pl_label_1, lw=3)
    
    pl_label_2 = 'Mead2016'
    ax1.loglog(ells, [ClWL_2(cur_ell, iz, iz) for cur_ell in ells], color=colors[6], ls='--', label=pl_label_2, lw=3)
    
    pl_label_3 = 'Mead2020'
    ax1.loglog(ells, [ClWL_3(cur_ell, iz, iz) for cur_ell in ells], color=colors[3], ls='--', label=pl_label_3, lw=3)
    
    pl_label_4 = 'Euclid Emulator'
    ax1.loglog(ells, [ClWL_4(cur_ell, iz, iz) for cur_ell in ells], color=colors[4], ls=':', label=pl_label_4, lw=3)
    
    pl_label_5 = 'BACCO Emulator'
    ax1.loglog(ells, [ClWL_5(cur_ell, iz, iz) for cur_ell in ells], color=colors[5], ls=':', label=pl_label_5, lw=3)
    
    ax1.set_xlabel(r'$\ell$', fontsize=26)
    ax1.set_ylabel(r'$C_\ell$ $[sr^{-1}]$', fontsize=26)
    ax1.set_title('bin i,j={:d},{:d}'.format(iz,iz))
    ax1.legend();

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(24,18), dpi=200)
fig.suptitle(r'Ratios of WL $C^{ij}_{\ell}$')

iz_dict={1:[0,0], 3:[0,1], 5:[1,0], 10:[1,1]}

for iz in [1,3,5,10]:
    
    ax1 = axs[iz_dict[iz][0],iz_dict[iz][1]]
      
    pl_label_2 = 'Mead2016'
    ax1.loglog(ells, [ClWL_2(cur_ell, iz, iz)/ClWL_1(cur_ell, iz, iz) for cur_ell in ells], color=colors[6], ls='--', label=pl_label_2, lw=3)
    
    pl_label_3 = 'Mead2020'
    ax1.loglog(ells, [ClWL_3(cur_ell, iz, iz)/ClWL_1(cur_ell, iz, iz) for cur_ell in ells], color=colors[3], ls='--', label=pl_label_3, lw=3)
    
    pl_label_4 = 'Euclid Emulator'
    ax1.loglog(ells, [ClWL_4(cur_ell, iz, iz)/ClWL_1(cur_ell, iz, iz) for cur_ell in ells], color=colors[4], ls=':', label=pl_label_4, lw=3)
    
    pl_label_5 = 'BACCO Emulator'
    ax1.loglog(ells, [ClWL_5(cur_ell, iz, iz)/ClWL_1(cur_ell, iz, iz) for cur_ell in ells], color=colors[5], ls=':', label=pl_label_5, lw=3)
    
    ax1.set_xlabel(r'$\ell$', fontsize=26)
    ax1.set_ylabel(r'$C_\ell/C_\ell^{\rm halofit}$', fontsize=26)
    ax1.set_title('bin i,j={:d},{:d}'.format(iz,iz))
    ax1.legend();

### Plotting the GC_phot C_ells

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(24,18), dpi=200)
fig.suptitle(r'GC photo $C^{ij}_{\ell}$')

iz_dict={1:[0,0], 3:[0,1], 5:[1,0], 10:[1,1]}

for iz in [1,3,5,10]:
    
    ax1 = axs[iz_dict[iz][0],iz_dict[iz][1]]
    
    pl_label_0 = 'Linear Recipe'
    ax1.loglog(ells, ClGC_0(ells, iz, iz), color=colors[1], ls='-.', label=pl_label_0, lw=3)
    
    pl_label_1 = 'Halofit'
    ax1.loglog(ells, ClGC_1(ells, iz, iz), color=colors[2], ls='--', label=pl_label_1, lw=3)
    
    pl_label_2 = 'Mead2016'
    ax1.loglog(ells, ClGC_2(ells, iz, iz), color=colors[6], ls='--', label=pl_label_2, lw=3)
    
    pl_label_3 = 'Mead2020'
    ax1.loglog(ells, ClGC_3(ells, iz, iz), color=colors[3], ls='--', label=pl_label_3, lw=3)
    
    pl_label_4 = 'Euclid Emulator'
    ax1.loglog(ells, ClGC_4(ells, iz, iz), color=colors[4], ls=':', label=pl_label_4, lw=3)

    pl_label_5 = 'BACCO Emulator'
    ax1.loglog(ells, ClGC_5(ells, iz, iz), color=colors[5], ls=':', label=pl_label_5, lw=3)
    
    ax1.set_xlabel(r'$\ell$', fontsize=26)
    ax1.set_ylabel(r'$C_\ell$ $[sr^{-1}]$', fontsize=26)
    ax1.set_title('bin i,j={:d},{:d}'.format(iz,iz))
    ax1.legend();

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(24,18), dpi=200)
fig.suptitle(r'Ratios GC photo $C^{ij}_{\ell}$')

iz_dict={1:[0,0], 3:[0,1], 5:[1,0], 10:[1,1]}

for iz in [1,3,5,10]:
    
    ax1 = axs[iz_dict[iz][0],iz_dict[iz][1]]
    
    pl_label_2 = 'Mead2016'
    ax1.loglog(ells, ClGC_2(ells, iz, iz)/ClGC_1(ells, iz, iz), color=colors[6], ls='--', label=pl_label_2, lw=3)
    
    pl_label_3 = 'Mead2020'
    ax1.loglog(ells, ClGC_3(ells, iz, iz)/ClGC_1(ells, iz, iz) , color=colors[3], ls='--', label=pl_label_3, lw=3)
    
    pl_label_4 = 'Euclid Emulator'
    ax1.loglog(ells, ClGC_4(ells, iz, iz)/ClGC_1(ells, iz, iz) , color=colors[4], ls=':', label=pl_label_4, lw=3)

    pl_label_5 = 'BACCO Emulator'
    ax1.loglog(ells, ClGC_5(ells, iz, iz)/ClGC_1(ells, iz, iz) , color=colors[5], ls=':', label=pl_label_5, lw=3)
    
    ax1.set_xlabel(r'$\ell$', fontsize=26)
    ax1.set_ylabel(r'$C_\ell/C_\ell^{\rm halofit}$', fontsize=26)
    ax1.set_title('bin i,j={:d},{:d}'.format(iz,iz))
    ax1.legend();

### Plotting the WL x GC photo Cl's

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(24,18), dpi=200)
fig.suptitle(r'WL x GC photo $C^{ij}_{\ell}$')

iz_dict={1:[0,0], 3:[0,1], 5:[1,0], 10:[1,1]}

for iz in [1,3,5,10]:
    
    ax1 = axs[iz_dict[iz][0],iz_dict[iz][1]]
    
    pl_label_0 = 'Linear Recipe'
    ax1.semilogx(ells, ClGCxWL_1(ells, iz, iz), color=colors[1], ls='-.', label=pl_label_0, lw=3)

    pl_label_1 = 'Halofit'
    ax1.semilogx(ells, ClGCxWL_1(ells, iz, iz), color=colors[2], ls='-.', label=pl_label_1, lw=3)

    pl_label_2 = 'Mead2016'
    ax1.semilogx(ells, ClGCxWL_2(ells, iz, iz), color=colors[6], ls='--', label=pl_label_2, lw=3)
    
    pl_label_3 = 'Mead2020'
    ax1.semilogx(ells, ClGCxWL_3(ells, iz, iz), color=colors[3], ls='--', label=pl_label_3, lw=3)
    
    pl_label_4 = 'Euclid Emulator'
    ax1.semilogx(ells, ClGCxWL_4(ells, iz, iz), color=colors[4], ls=':', label=pl_label_4, lw=3)

    pl_label_5 = 'BACCO Emulator'
    ax1.semilogx(ells, ClGCxWL_5(ells, iz, iz), color=colors[5], ls=':', label=pl_label_5, lw=3)
    
    ax1.set_xlabel(r'$\ell$', fontsize=26)
    ax1.set_ylabel(r'$C_\ell$ $[sr^{-1}]$', fontsize=26)
    ax1.set_title('bin i,j={:d},{:d}'.format(iz,iz))
    ax1.legend();

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(24,18), dpi=200)
fig.suptitle(r'WL x GC photo $C^{ij}_{\ell}$')

iz_dict={1:[0,0], 3:[0,1], 5:[1,0], 10:[1,1]}

for iz in [1,3,5,10]:
    
    ax1 = axs[iz_dict[iz][0],iz_dict[iz][1]]

    pl_label_2 = 'Mead2016'
    ax1.semilogx(ells, ClGCxWL_2(ells, iz, iz)/ClGCxWL_1(ells, iz, iz) , color=colors[6], ls='--', label=pl_label_2, lw=3)
    
    pl_label_3 = 'Mead2020'
    ax1.semilogx(ells, ClGCxWL_3(ells, iz, iz)/ClGCxWL_1(ells, iz, iz) , color=colors[3], ls='--', label=pl_label_3, lw=3)
    
    pl_label_4 = 'Euclid Emulator'
    ax1.semilogx(ells, ClGCxWL_4(ells, iz, iz)/ClGCxWL_1(ells, iz, iz) , color=colors[4], ls=':', label=pl_label_4, lw=3)

    pl_label_5 = 'BACCO Emulator'
    ax1.semilogx(ells, ClGCxWL_5(ells, iz, iz)/ClGCxWL_1(ells, iz, iz) , color=colors[5], ls=':', label=pl_label_5, lw=3)
    
    ax1.set_xlabel(r'$\ell$', fontsize=26)
    ax1.set_ylabel(r'$C_\ell/C_\ell^{\rm halofit}$', fontsize=26)
    ax1.set_title('bin i,j={:d},{:d}'.format(iz,iz))
    ax1.legend();

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(30,20))
fig.tight_layout(h_pad=5,w_pad=5)
#fig.suptitle(r'Ratios WL $C_{\ell}$')

i_WL = 10
i_GC = 5

buffer, err = get_error_bars(i_WL,i_WL,probe='WL')
ells_d = ells[ells<=buffer[-1]]

power_label = 'E{:d}-E{:d}'.format(i_WL, i_WL)
Cl_d = read_data.data_dict['WL'][power_label]

axs[0,1].semilogx(ells_d, ells_d**0, ls='--', lw=5)

pl_label_2 = 'HMcode2020'
axs[0,1].semilogx(ells_d, ClWL_3(ells_d, i_WL, i_WL)/ClWL_1(ells_d, i_WL, i_WL) , ls='-', label=pl_label_2, lw=5)

pl_label_2 = 'EE2'
axs[0,1].semilogx(ells_d, ClWL_4(ells_d, i_WL, i_WL)/ClWL_1(ells_d, i_WL, i_WL) , ls='-', label=pl_label_2, lw=5)

pl_label_4 = 'Bacco'
axs[0,1].semilogx(ells_d, ClWL_5(ells_d, i_WL, i_WL)/ClWL_1(ells_d, i_WL, i_WL) , ls='-', label=pl_label_4, lw=5)

pl_label_2 = 'HMcode2016'
axs[0,1].semilogx(ells_d, ClWL_2(ells_d, i_WL, i_WL)/ClWL_1(ells_d, i_WL, i_WL) , ls='-', label=pl_label_2, lw=5)


axs[0,1].fill_between(buffer, 1+err/Cl_d,1-err/Cl_d,alpha = 0.15, color = 'b')

axs[0,1].set_xlabel(r'$\ell$', fontsize=40)
axs[0,1].set_ylabel(r'WL $C_\ell/C_\ell^{\rm Halofit}$', fontsize=40)
axs[0,1].set_title('WL bin i,j={:d},{:d}'.format(i_WL,i_WL), fontsize=40)


axs[0,1].set_ylim([0.94,1.06])
axs[0,1].set_xlim([buffer[0],buffer[-1]])
#axs[0,1].legend();
axs[0,1].tick_params(axis='both', which='major', labelsize=40)


buffer, err = get_error_bars(i_GC,i_GC)
ells_d = ells[ells<=buffer[-1]]
power_label = 'P{:d}-P{:d}'.format(i_GC, i_GC)
Cl_d = read_data.data_dict['GC-Phot'][power_label]

axs[1,0].semilogx(ells_d, ells_d**0, ls='--', lw=5)

pl_label_2 = 'HMcode2020'
axs[1,0].semilogx(ells_d, ClGC_3(ells_d, i_GC, i_GC)/ClGC_1(ells_d, i_GC, i_GC) , ls='-', label=pl_label_2, lw=5)

pl_label_2 = 'EE2'
axs[1,0].semilogx(ells_d, ClGC_4(ells_d, i_GC, i_GC)/ClGC_1(ells_d, i_GC, i_GC) , ls='-', label=pl_label_2, lw=5)

pl_label_4 = 'Bacco'
axs[1,0].semilogx(ells_d, ClGC_5(ells_d, i_GC, i_GC)/ClGC_1(ells_d, i_GC, i_GC) , ls='-', label=pl_label_4, lw=5)

pl_label_2 = 'HMcode2016'
axs[1,0].semilogx(ells_d, ClGC_2(ells_d, i_GC, i_GC)/ClGC_1(ells_d, i_GC, i_GC) , ls='-', label=pl_label_2, lw=5)

axs[1,0].fill_between(buffer, 1+err/Cl_d,1-err/Cl_d,alpha = 0.15, color = 'b')

axs[1,0].set_xlabel(r'$\ell$', fontsize=40)
axs[1,0].set_ylabel(r'GC $C_\ell/C_\ell^{\rm Halofit}$', fontsize=40)
axs[1,0].set_title('GCphot bin i,j={:d},{:d}'.format(i_GC,i_GC), fontsize=40)


axs[1,0].set_ylim([0.94,1.06])
axs[1,0].set_xlim([buffer[0],buffer[-1]])
#axs[1,0].legend();
axs[1,0].tick_params(axis='both', which='major', labelsize=40)

buffer, err = get_error_bars_XC(i_GC,i_WL)
ells_d = ells[ells<=buffer[-1]]
power_label = 'P{:d}-E{:d}'.format(i_GC, i_WL)
Cl_d = read_data.data_dict['XC-Phot'][power_label]

axs[1,1].semilogx(ells_d, ells_d**0, ls='--', lw=5)

pl_label_2 = 'HMcode2020'
axs[1,1].semilogx(ells_d, ClGCxWL_3(ells_d, i_GC, i_WL)/ClGCxWL_1(ells_d, i_GC, i_WL) , ls='-', label=pl_label_2, lw=5)

pl_label_2 = 'EE2'
axs[1,1].semilogx(ells_d, ClGCxWL_4(ells_d, i_GC, i_WL)/ClGCxWL_1(ells_d, i_GC, i_WL) , ls='-', label=pl_label_2, lw=5)

pl_label_4 = 'Bacco'
axs[1,1].semilogx(ells_d, ClGCxWL_5(ells_d, i_GC, i_WL)/ClGCxWL_1(ells_d, i_GC, i_WL) , ls='-', label=pl_label_4, lw=5)

pl_label_2 = 'HMcode2016'
axs[1,1].semilogx(ells_d, ClGCxWL_2(ells_d, i_GC, i_WL)/ClGCxWL_1(ells_d, i_GC, i_WL) , ls='-', label=pl_label_2, lw=5)

axs[1,1].fill_between(buffer, 1+err/Cl_d,1-err/Cl_d,alpha = 0.15, color = 'b')

axs[1,1].set_xlabel(r'$\ell$', fontsize=40)
axs[1,1].set_ylabel(r'GCxWL $C_\ell/C_\ell^{\rm Halofit}$', fontsize=40)
axs[1,1].set_title('GCphot x WL bin i,j={:d},{:d}'.format(i_GC,i_WL), fontsize=40)


axs[1,1].set_ylim([0.94,1.06])
axs[1,1].set_xlim([buffer[0],buffer[-1]])

#axs[1,1].legend();
axs[1,1].tick_params(axis='both', which='major', labelsize=40)

axs[0,0].semilogx(wavenumbers, wavenumbers**0, ls='--', lw=5)

pl_label_2 = 'HMcode2020'
axs[0,0].semilogx(wavenumbers, cosmo_dic_3['Pmm_phot'](0.8445, wavenumbers).flatten()/cosmo_dic_1['Pmm_phot'](0.8445, wavenumbers).flatten(), ls='-', label=pl_label_2, lw=5)

pl_label_2 = 'EE2'
axs[0,0].semilogx(wavenumbers, cosmo_dic_4['Pmm_phot'](0.8445, wavenumbers).flatten()/cosmo_dic_1['Pmm_phot'](0.8445, wavenumbers).flatten(), ls='-', label=pl_label_2, lw=5)

pl_label_4 = 'Bacco'
axs[0,0].semilogx(wavenumbers, cosmo_dic_5['Pmm_phot'](0.8445, wavenumbers).flatten()/cosmo_dic_1['Pmm_phot'](0.8445, wavenumbers).flatten(), ls='-', label=pl_label_4, lw=5)

pl_label_2 = 'HMcode2016'
axs[0,0].semilogx(wavenumbers, cosmo_dic_2['Pmm_phot'](0.8445, wavenumbers).flatten()/cosmo_dic_1['Pmm_phot'](0.8445, wavenumbers).flatten(), ls='-', label=pl_label_2, lw=5)


#axs[0,0].fill_between(wavenumbers, 1+0.01*wavenumbers**0,1-0.01*wavenumbers**0,alpha = 0.15, color = 'b')
#axs[0,0].fill_between(wavenumbers, 1+0.02*wavenumbers**0,1-0.02*wavenumbers**0,alpha = 0.15, color = 'b')
axs[0,0].set_ylim([0.94,1.06])
axs[0,0].set_title(r'$P_{mm}(k)\ z=0.8445$', fontsize=40)
axs[0,0].set_xlabel(r'$k$ $[{Mpc}^{-1}]$', fontsize=40)
axs[0,0].set_ylabel(r'$P/P^{\rm Halofit}$', fontsize=40)
axs[0,0].set_xlim([1e-3,10])
axs[0,0].tick_params(axis='both', which='major', labelsize=40)


axs[0,0].legend(loc=2,fontsize=40);


## Baryon testing

In [ ]:
like.cosmo.cosmo_dic['NL_flag_phot_matter'] = 4

# HMcode2016 baryons
like_mead16f.cosmo.cosmo_dic['NL_flag_phot_baryon'] = 1
like_mead16f.cosmo.update_cosmo_dic(like_mead16f.cosmo.cosmo_dic['z_win'], wavenumber_growth)
cosmo_dic_2_b_1 = copy.copy(like_mead16f.cosmo.cosmo_dic)

# HMcode2020 baryons
like_mead20fb.cosmo.cosmo_dic['NL_flag_phot_matter'] = 3
like_mead20fb.cosmo.cosmo_dic['NL_flag_phot_baryon'] = 2
like_mead20fb.cosmo.update_cosmo_dic(like_mead20fb.cosmo.cosmo_dic['z_win'], wavenumber_growth)
cosmo_dic_3_b_2 = copy.copy(like_mead20fb.cosmo.cosmo_dic)

# BCemu baryons
like.cosmo.cosmo_dic['NL_flag_phot_baryon'] = 3
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], wavenumber_growth)
cosmo_dic_4_b_3 = copy.copy(like.cosmo.cosmo_dic)

# Bacco baryons
like.cosmo.cosmo_dic['NL_flag_phot_baryon'] = 4
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], wavenumber_growth)
cosmo_dic_4_b_4 = copy.copy(like.cosmo.cosmo_dic)

### Plot Pmm

In [ ]:
## Create four panels, one for each of the chosen redshifts above

fig, axs = plt.subplots(2, 2, figsize=(24,18))
fig.suptitle('Spectra')

ax=axs[0,0]

Pab = 'Pmm_phot'


for iz in range(4):
    if iz==0:
        ax=axs[0,0]
    if iz==1:
        ax=axs[0,1]
    if iz==2:
        ax=axs[1,0]
    if iz==3:
        ax=axs[1,1]
    z0=redshift_chosen[iz]
    
    
    ax.loglog(wavenumbers, cosmo_dic_0[Pab](z0, wavenumbers).flatten(), '--', 
               label=r"Linear recipe z={:.2f}".format(z0), c=colors[1])
    
    #------
    ax.loglog(wavenumbers, cosmo_dic_4[Pab](z0, wavenumbers).flatten(), '--', 
               label=r"EE2, no baryons z={:.2f}".format(z0), c=colors[2])
    
    #---
    ax.loglog(wavenumbers, cosmo_dic_4_b_4[Pab](z0, wavenumbers).flatten(), '.-', 
               label=r"EE2, bacco baryons z={:.2f}".format(z0), c=colors[4])
    
    ax.loglog(wavenumbers, cosmo_dic_4_b_3[Pab](z0, wavenumbers).flatten(), '.-', 
               label=r"EE2, BCemu baryons z={:.2f}".format(z0), c=colors[5])

    ax.loglog(wavenumbers, cosmo_dic_3_b_2[Pab](z0, wavenumbers).flatten(), '.-', 
               label=r"HM20, HM20 baryons z={:.2f}".format(z0), c=colors[6])
    
    ax.loglog(wavenumbers, cosmo_dic_2_b_1[Pab](z0, wavenumbers).flatten(), '.-', 
               label=r"HM16, HM16 baryons z={:.2f}".format(z0), c=colors[7]) 
    
    ax.set_xlabel(r'$k$ $[{Mpc}^{-1}]$', fontsize=24)
    ax.set_ylabel(r'$P_{mm}^{phot} [{Mpc}^{-3}]$')
    ax.legend(frameon=False, fontsize=24)

In [ ]:
## Now the ratios

fig, axs = plt.subplots(2, 2, figsize=(24,18))
fig.suptitle('Suppresion')


ax=axs[0,0]

Pab = 'Pmm_phot'


for iz in range(4):
    if iz==0:
        ax=axs[0,0]
    if iz==1:
        ax=axs[0,1]
    if iz==2:
        ax=axs[1,0]
    if iz==3:
        ax=axs[1,1]
    z0=redshift_chosen[iz]
    
    #---
    
    ax.semilogx(wavenumbers, cosmo_dic_4[Pab](z0, wavenumbers).flatten()/cosmo_dic_4[Pab](z0, wavenumbers).flatten(), '--', 
               label=r"EE2 z={:.2f}".format(z0), c=colors[3])
    
    #
    ax.semilogx(wavenumbers, cosmo_dic_4_b_4[Pab](z0, wavenumbers).flatten()/cosmo_dic_4[Pab](z0, wavenumbers).flatten(), '.-', 
               label=r"EE2 Bacco z={:.2f}".format(z0), c=colors[4])
    
    ax.semilogx(wavenumbers, cosmo_dic_4_b_3[Pab](z0, wavenumbers).flatten()/cosmo_dic_4[Pab](z0, wavenumbers).flatten(), '.-', 
               label=r"EE2 BCemu z={:.2f}".format(z0), c=colors[5])
    
    ax.semilogx(wavenumbers, cosmo_dic_3_b_2[Pab](z0, wavenumbers).flatten()/cosmo_dic_3[Pab](z0, wavenumbers).flatten(), '.-', 
               label=r"HM20 z={:.2f}".format(z0), c=colors[6])
    
    ax.semilogx(wavenumbers, cosmo_dic_2_b_1[Pab](z0, wavenumbers).flatten()/cosmo_dic_2[Pab](z0, wavenumbers).flatten(), '.-', 
               label=r"HM16 z={:.2f}".format(z0), c=colors[7])
    
    ax.axvline(5 * cosmo_dic_3['H0']/100, c='k', ls =':', lw=1)
    
    ax.set_xlabel(r'$k$ $[{Mpc}^{-1}]$', fontsize=24)
    ax.set_ylabel(r'$P_{mm}/P_{mm}^{\rm NoBar} [{Mpc}^{-3}]$')
    
    #ax.set_ylim([0.9,1.1])
    
    ax.legend(frameon=False, fontsize=24)

In [ ]:
phot_ins_4_b_4 = Photo( cosmo_dic_4_b_4, read_data.nz_dict_WL,
                      read_data.nz_dict_GC_Phot)
phot_ins_4_b_3 = Photo( cosmo_dic_4_b_3, read_data.nz_dict_WL,
                      read_data.nz_dict_GC_Phot)
phot_ins_3_b_2 = Photo( cosmo_dic_3_b_2, read_data.nz_dict_WL,
                      read_data.nz_dict_GC_Phot)
phot_ins_2_b_1 = Photo( cosmo_dic_2_b_1, read_data.nz_dict_WL,
                      read_data.nz_dict_GC_Phot)

In [ ]:
ClGC_2_b_1 = phot_ins_2_b_1.Cl_GC_phot
ClGC_3_b_2 = phot_ins_3_b_2.Cl_GC_phot
ClGC_4_b_3 = phot_ins_4_b_3.Cl_GC_phot
ClGC_4_b_4 = phot_ins_4_b_4.Cl_GC_phot

In [ ]:
ClWL_2_b_1 = phot_ins_2_b_1.Cl_WL
ClWL_3_b_2 = phot_ins_3_b_2.Cl_WL
ClWL_4_b_3 = phot_ins_4_b_3.Cl_WL
ClWL_4_b_4 = phot_ins_4_b_4.Cl_WL

In [ ]:
ClGCxWL_2_b_1 = phot_ins_2_b_1.Cl_cross
ClGCxWL_3_b_2 = phot_ins_3_b_2.Cl_cross
ClGCxWL_4_b_3 = phot_ins_4_b_3.Cl_cross
ClGCxWL_4_b_4 = phot_ins_4_b_4.Cl_cross

### Plotting the WL C_ells

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(24,18), dpi=200)
fig.suptitle(r'WL $C^{ij}_{\ell}$')

iz_dict={1:[0,0], 3:[0,1], 5:[1,0], 10:[1,1]}

for iz in [1,3,5,10]:
    
    ax1 = axs[iz_dict[iz][0],iz_dict[iz][1]]
    
    pl_label_0 = 'Linear'
    ax1.loglog(ells, ClWL_0(ells, iz, iz) , color=colors[1], ls='-.', label=pl_label_0, lw=3)
    
    pl_label_3 = 'No bar'
    ax1.loglog(ells, ClWL_4(ells, iz, iz) , color=colors[2], ls=':', label=pl_label_3, lw=3)
    
    pl_label_4 = 'Bacco'
    ax1.loglog(ells, ClWL_4_b_4(ells, iz, iz) , color=colors[3], ls=':', label=pl_label_4, lw=3)

    pl_label_4 = 'BCemu'
    ax1.loglog(ells, ClWL_4_b_3(ells, iz, iz) , color=colors[4], ls=':', label=pl_label_4, lw=3)

    pl_label_5 = 'HM20'
    ax1.loglog(ells, ClWL_3_b_2(ells, iz, iz) , color=colors[5], ls=':', label=pl_label_5, lw=3)

    pl_label_6 = 'HM16'
    ax1.loglog(ells, ClWL_2_b_1(ells, iz, iz) , color=colors[6], ls=':', label=pl_label_6, lw=3)

    ax1.set_xlabel(r'$\ell$', fontsize=26)
    ax1.set_ylabel(r'$C_\ell$ $[sr^{-1}]$', fontsize=26)
    ax1.set_title('bin i,j={:d},{:d}'.format(iz,iz))
    ax1.legend();

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(24,18), dpi=200)
fig.suptitle(r'Ratios of WL $C^{ij}_{\ell}$')

iz_dict={1:[0,0], 3:[0,1], 5:[1,0], 10:[1,1]}

for iz in [1,3,5,10]:
    
    ax1 = axs[iz_dict[iz][0],iz_dict[iz][1]]
      
    pl_label_2 = 'No bar'
    ax1.loglog(ells, ClWL_4(ells, iz, iz)/ClWL_4(ells, iz, iz) , color=colors[3], ls='--', label=pl_label_2, lw=3)
    
    pl_label_3 = 'Bacco'
    ax1.loglog(ells, ClWL_4_b_4(ells, iz, iz)/ClWL_4(ells, iz, iz) , color=colors[4], ls=':', label=pl_label_3, lw=3)
    
    pl_label_4 = 'BCemu'
    ax1.loglog(ells, ClWL_4_b_3(ells, iz, iz)/ClWL_4(ells, iz, iz) , color=colors[5], ls=':', label=pl_label_4, lw=3)

    pl_label_5 = 'HM20'
    ax1.loglog(ells, ClWL_3_b_2(ells, iz, iz)/ClWL_3(ells, iz, iz) , color=colors[6], ls=':', label=pl_label_5, lw=3)

    pl_label_6 = 'HM16'
    ax1.loglog(ells, ClWL_2_b_1(ells, iz, iz)/ClWL_2(ells, iz, iz) , color=colors[7], ls=':', label=pl_label_6, lw=3)

    
    ax1.set_xlabel(r'$\ell$', fontsize=26)
    ax1.set_ylabel(r'$C_\ell/C_\ell^{\rm No bar}$', fontsize=26)
    ax1.set_title('bin i,j={:d},{:d}'.format(iz,iz))
    ax1.legend();

### Plotting the GC C_ells

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(24,18), dpi=200)
fig.suptitle(r'GC $C^{ij}_{\ell}$')

iz_dict={1:[0,0], 3:[0,1], 5:[1,0], 10:[1,1]}

for iz in [1,3,5,10]:
    
    ax1 = axs[iz_dict[iz][0],iz_dict[iz][1]]
    
    pl_label_0 = 'Linear'
    ax1.loglog(ells, ClGC_0(ells, iz, iz) , color=colors[1], ls='-.', label=pl_label_0, lw=3)
    
    pl_label_3 = 'No bar'
    ax1.loglog(ells, ClGC_4(ells, iz, iz) , color=colors[2], ls=':', label=pl_label_3, lw=3)
    
    pl_label_4 = 'Bacco'
    ax1.loglog(ells, ClGC_4_b_4(ells, iz, iz) , color=colors[3], ls=':', label=pl_label_4, lw=3)

    pl_label_4 = 'BCemu'
    ax1.loglog(ells, ClGC_4_b_3(ells, iz, iz) , color=colors[4], ls=':', label=pl_label_4, lw=3)
    
    pl_label_4 = 'HM20'
    ax1.loglog(ells, ClGC_3_b_2(ells, iz, iz) , color=colors[5], ls=':', label=pl_label_4, lw=3)

    pl_label_4 = 'HM16'
    ax1.loglog(ells, ClGC_2_b_1(ells, iz, iz) , color=colors[6], ls=':', label=pl_label_4, lw=3)

    
    ax1.set_xlabel(r'$\ell$', fontsize=26)
    ax1.set_ylabel(r'$C_\ell$ $[sr^{-1}]$', fontsize=26)
    ax1.set_title('bin i,j={:d},{:d}'.format(iz,iz))
    ax1.legend();

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(24,18), dpi=200)
fig.suptitle(r'Ratios of GC $C^{ij}_{\ell}$')

iz_dict={1:[0,0], 3:[0,1], 5:[1,0], 10:[1,1]}

for iz in [1,3,5,10]:
    
    ax1 = axs[iz_dict[iz][0],iz_dict[iz][1]]
      
    pl_label_2 = 'No bar'
    ax1.loglog(ells, ClGC_4(ells, iz, iz)/ClGC_4(ells, iz, iz) , color=colors[3], ls='--', label=pl_label_2, lw=3)
    
    pl_label_3 = 'Bacco'
    ax1.loglog(ells, ClGC_4_b_4(ells, iz, iz)/ClGC_4(ells, iz, iz) , color=colors[4], ls=':', label=pl_label_3, lw=3)
    
    pl_label_4 = 'BCemu'
    ax1.loglog(ells, ClGC_4_b_3(ells, iz, iz)/ClGC_4(ells, iz, iz) , color=colors[5], ls=':', label=pl_label_4, lw=3)
    
    pl_label_4 = 'HM20'
    ax1.loglog(ells, ClGC_3_b_2(ells, iz, iz)/ClGC_3(ells, iz, iz) , color=colors[6], ls=':', label=pl_label_4, lw=3)
    
    pl_label_4 = 'HM16'
    ax1.loglog(ells, ClGC_2_b_1(ells, iz, iz)/ClGC_2(ells, iz, iz) , color=colors[7], ls=':', label=pl_label_4, lw=3)
    
    ax1.set_xlabel(r'$\ell$', fontsize=26)
    ax1.set_ylabel(r'$C_\ell/C_\ell^{\rm No bar}$', fontsize=26)
    ax1.set_title('bin i,j={:d},{:d}'.format(iz,iz))
    ax1.legend();

### Plotting the WLxGC C_ells

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(24,18), dpi=200)
fig.suptitle(r'GCxWL $C^{ij}_{\ell}$')

iz_dict={1:[0,0], 3:[0,1], 5:[1,0], 10:[1,1]}

for iz in [1,3,5,10]:
    
    ax1 = axs[iz_dict[iz][0],iz_dict[iz][1]]
    
    pl_label_0 = 'Linear'
    ax1.semilogx(ells, ClGCxWL_0(ells, iz, iz) , color=colors[1], ls='-.', label=pl_label_0, lw=3)
    
    pl_label_3 = 'No bar'
    ax1.semilogx(ells, ClGCxWL_4(ells, iz, iz) , color=colors[2], ls=':', label=pl_label_3, lw=3)
    
    pl_label_4 = 'Bacco'
    ax1.semilogx(ells, ClGCxWL_4_b_4(ells, iz, iz) , color=colors[3], ls=':', label=pl_label_4, lw=3)

    pl_label_4 = 'BCemu'
    ax1.semilogx(ells, ClGCxWL_4_b_3(ells, iz, iz) , color=colors[4], ls=':', label=pl_label_4, lw=3)

    pl_label_4 = 'HM20'
    ax1.semilogx(ells, ClGCxWL_3_b_2(ells, iz, iz) , color=colors[5], ls=':', label=pl_label_4, lw=3)

    pl_label_4 = 'HM16'
    ax1.semilogx(ells, ClGCxWL_2_b_1(ells, iz, iz) , color=colors[6], ls=':', label=pl_label_4, lw=3)


    ax1.set_xlabel(r'$\ell$', fontsize=26)
    ax1.set_ylabel(r'$C_\ell$ $[sr^{-1}]$', fontsize=26)
    ax1.set_title('bin i,j={:d},{:d}'.format(iz,iz))
    ax1.legend();

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(24,18), dpi=200)
fig.suptitle(r'Ratios of GCxWL $C^{ij}_{\ell}$')

iz_dict={1:[0,0], 3:[0,1], 5:[1,0], 10:[1,1]}

for iz in [1,3,5,10]:
    
    ax1 = axs[iz_dict[iz][0],iz_dict[iz][1]]
      
    pl_label_2 = 'No bar'
    ax1.semilogx(ells, ClGCxWL_4(ells, iz, iz)/ClGCxWL_4(ells, iz, iz) , color=colors[3], ls='--', label=pl_label_2, lw=3)
    
    pl_label_3 = 'Bacco'
    ax1.semilogx(ells, ClGCxWL_4_b_4(ells, iz, iz)/ClGCxWL_4(ells, iz, iz) , color=colors[4], ls=':', label=pl_label_3, lw=3)
    
    pl_label_4 = 'BCemu'
    ax1.semilogx(ells, ClGCxWL_4_b_3(ells, iz, iz)/ClGCxWL_4(ells, iz, iz) , color=colors[5], ls=':', label=pl_label_4, lw=3)

    pl_label_5 = 'HM20'
    ax1.semilogx(ells, ClGCxWL_3_b_2(ells, iz, iz)/ClGCxWL_3(ells, iz, iz) , color=colors[6], ls=':', label=pl_label_5, lw=3)

    pl_label_5 = 'HM16'
    ax1.semilogx(ells, ClGCxWL_2_b_1(ells, iz, iz)/ClGCxWL_2(ells, iz, iz) , color=colors[7], ls=':', label=pl_label_5, lw=3)


    ax1.set_xlabel(r'$\ell$', fontsize=26)
    ax1.set_ylabel(r'$C_\ell/C_\ell^{\rm No bar}$', fontsize=26)
    ax1.set_title('bin i,j={:d},{:d}'.format(iz,iz))
    ax1.legend();

## Timing all options

In [ ]:
# First copy the info dict 
info_speed = copy.deepcopy(info)

# We measure the time it takes for a posterior evaluation. Arbitrarily, we choose to sample ns.
info_speed['params']['ns'] = {'prior':{'min':0.95, 'max':0.97}}

# We start with NL_flag_phot_matter = 0
info_speed['likelihood']['Euclid']['NL_flag_phot_matter'] = 0
info_speed['likelihood']['Euclid']['NL_flag_phot_baryon'] = 0

set_halofit_version(info_speed, info_speed['likelihood']['Euclid']['NL_flag_phot_matter'],
                               info_speed['likelihood']['Euclid']['NL_flag_phot_baryon'])

model_speed = get_model(info_speed)

# Evaluate posterior once to ensure that everything is loaded before measuring time
point = dict(zip(model_speed.parameterization.sampled_params(),
                 model_speed.prior.sample(ignore_external=True)[0]))
logposterior = model_speed.logposterior(point)

# Evalute posterior n_eval times and average

n_eval = 10

t1 = time.time()

for i in range(n_eval):
    point = dict(zip(model_speed.parameterization.sampled_params(),
                     model_speed.prior.sample(ignore_external=True)[0]))

    logposterior = model_speed.logposterior(point)

t2 = time.time()

t_ave = (t2 - t1) / n_eval

print("\nAverage time for posterior evaluation for NL_flag_phot_matter =", 
      info_speed['likelihood']['Euclid']['NL_flag_phot_matter'],
      "is", t_ave, "seconds.")

In [ ]:
t_nl = [[t_ave, t_ave, t_ave, t_ave, t_ave]]

for NL_f in [1,2,3,4,5]:
    
    t_bar = []
    for B_f in [0,1,2,3,4]:

        info_speed['likelihood']['Euclid']['NL_flag_phot_matter'] = NL_f
        info_speed['likelihood']['Euclid']['NL_flag_phot_baryon'] = B_f

        set_halofit_version(info_speed, info_speed['likelihood']['Euclid']['NL_flag_phot_matter'],
                                       info_speed['likelihood']['Euclid']['NL_flag_phot_baryon'])

        model_speed = get_model(info_speed)

        # Evaluate posterior once to ensure that everything is loaded before measuring time
        point = dict(zip(model_speed.parameterization.sampled_params(),
                         model_speed.prior.sample(ignore_external=True)[0]))
        logposterior = model_speed.logposterior(point)

        # Evalute posterior n_eval times and average

        n_eval = 10

        t1 = time.time()

        for i in range(n_eval):
            point = dict(zip(model_speed.parameterization.sampled_params(),
                             model_speed.prior.sample(ignore_external=True)[0]))

            logposterior = model_speed.logposterior(point)

        t2 = time.time()
        
        t_bar.append((t2-t1)/n_eval)

        print("\nAverage time for NL_flag=",NL_f,
              "and Bar_flag=",B_f,"is",(t2-t1)/n_eval,"seconds.")
        
    t_nl.append(t_bar)
    
t_nl = np.asarray(t_nl)

In [ ]:
plt.matshow(t_nl,cmap='coolwarm')

for i in range(5):
    for j in range(6):
        c = t_nl[j, i].round(2)
        if not (j==0 and i!=2):
            plt.text(i, j, str(c), va='center', ha='center',size=22)

plt.xticks(np.arange(5), ['NoBar', 'HM16', 'HM20', 'BCemu', 'Bacco'],size=15)
plt.yticks(np.arange(6), ['All Linear', 'Halofit', 'HM16', 'HM20', 'EE2', 'Bacco'],size=15)